In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
from sklearn.decomposition import PCA
from scipy.spatial import distance
from scipy.spatial import distance_matrix

# Create output folders

In [2]:
input_path_spatial_graphs = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Input_Spatial_Graphs/'
input_path_avg_rf = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Input_Ref_Frame/'
input_path_layer_borders_v1 = input_path_avg_rf+'Layer_Borders_v1/'
input_path_layer_borders_v2 = input_path_avg_rf+'Layer_Borders_v2/'
manual_neuron_axis_path = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Manual_Neuron_Axis/'
manual_cell_type_assignment = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/vm1_OPTICS_mastersheet_afterMO.xlsx'
layer_borders_v1 = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/vM1_Layer_Borders_Per_Depths.csv'

surface_resolution = 100

ref_barrels = AmiraSpatialGraph(input_path_avg_rf+'Avg_SBF_Without_Projections.am',generic_graph=True,\
                                barrel_projections_present=False)
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_50.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_50.vtk')
ref_pia_wm = Surface(polydata=ref_pia.surface)
ref_pia_wm.append(ref_wm.surface)

coronal_pia = Surface(input_path_avg_rf+'MG48_lhs_pia_open_bottom_voxel_size_50.vtk')
coronal_wm = Surface(input_path_avg_rf+'MG48_lhs_WM_open_bottom_voxel_size_50.vtk')
coronal_pia_wm = Surface(polydata=coronal_pia.surface)
coronal_pia_wm.append(coronal_wm.surface)



In [3]:
# create output folder structure
HOME = str(pathlib.Path(input_path_spatial_graphs).parent)
output_root = HOME + '/Outputs/'
pathlib.Path((output_root)).mkdir(exist_ok=True)

output_surfaces = output_root + '1_Surfaces/'
pathlib.Path((output_surfaces)).mkdir(exist_ok=True)

output_axis_field = output_root + '1_Axis_Fields/'
pathlib.Path((output_axis_field)).mkdir(exist_ok=True)

output_stats = output_root + '2_Local_Stats/'
pathlib.Path((output_stats)).mkdir(exist_ok=True)

output_neuron_axis = output_root + '2_Local_Neuron_Axis/'
pathlib.Path((output_neuron_axis)).mkdir(exist_ok=True)

output_neuron_axis_manual = output_root + '2_Manual_Neuron_Axis/'
pathlib.Path((output_neuron_axis_manual)).mkdir(exist_ok=True)

output_alignment_using_surfaces = output_root + '3_Aligned_By_Surfaces/'
pathlib.Path((output_alignment_using_surfaces)).mkdir(exist_ok=True)

output_alignment_using_barrels = output_root + '3_Aligned_By_Barrels/'
pathlib.Path((output_alignment_using_barrels)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_nearest_rel_depth = output_root + '4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_nearest_rel_depth)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_local_avg_rel_depth = output_root + '4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_local_avg_rel_depth)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_nearest_pia_soma = output_root + '4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Pia_Soma_Dist_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_nearest_pia_soma)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_local_Avg_pia_soma = output_root + '4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_local_Avg_pia_soma)).mkdir(exist_ok=True)

output_alignment_final = output_root + '4_Final_Registration/'
pathlib.Path((output_alignment_final)).mkdir(exist_ok=True)

for file in glob.glob(manual_neuron_axis_path+'*'):
    shutil.copyfile(file,output_neuron_axis_manual+os.path.basename(file))

output_canonical = HOME + '/Canonical_View/'
pathlib.Path((output_canonical)).mkdir(exist_ok=True)

output_standardization = output_root + '5_Standardized_Neurons/'
pathlib.Path((output_standardization)).mkdir(exist_ok=True)

# Helper Functions

In [23]:
# rotation from a to b
def align_a_to_b(a,b):
    u_a = Vectors().get_unit_vec(a[0],a[1])
    u_b = Vectors().get_unit_vec(b[0],b[1])
    v = np.cross(u_a,u_b)
    s = np.linalg.norm(v)
    c = np.dot(u_a,u_b)
    I = np.array([1,0,0,0,1,0,0,0,1]).reshape([3,3])
    #print(I)
    v_x = np.array([0,-v[2],v[1],v[2],0,-v[0],-v[1],v[0],0]).reshape([3,3])
    R = I + v_x + np.dot(v_x,v_x) * (1/(1+c))
    r_list = []
    for i in range(3):
        for j in range(3):
            r_list.append(R[i,j])
    #print(R)
    return R

In [24]:
def convert_to_transformation_matrix(rot_mat,translation):
    
    tr_mat = []
    k = 0

    for i in range(3):
        for j in range(3):
            tr_mat.append(rot_mat[i,j])
            k = k + 1
        tr_mat.append(0)
        k = k+1
    tr_mat.append(-translation[0])
    tr_mat.append(-translation[1])
    tr_mat.append(-translation[2])
    tr_mat.append(1)
    return tr_mat

In [ ]:
def write_edge(edge,filename):
    axis_sg = AmiraSpatialGraph(generic_graph=True)
    axis_sg.graph_data.add_edge(edge[0],edge[1])
    axis_sg.write_spatial_graph(filename)

    return axis_sg

    

In [ ]:
def resample_edge(edge,num_pts_to_be_inserted):
    resampled_edge = []
    #resample_edge.append(edge[0])
    n = np.array(edge[len(edge)-1]) - np.array(edge[0])
    n_unit = n / np.linalg.norm(n)
    edge_len = Vectors().get_vec_length(edge)
    for i in range(num_pts_to_be_inserted):
        resampled_edge.append(i*n_unit*edge_len/num_pts_to_be_inserted+edge[0])
    return edge

In [ ]:
def get_pca_direction(apical_dendrite_pts,res=1,two_d_only=False):
#     resampled_pts = []
#     for edge in apical_dendrite_edges:
#         for i in range(len(edge)-1):
#             length = distance.euclidean(edge[i],edge[i+1])
#             if length > res:
#                 num_pts_to_be_inserted = int(length/res)
#                 resampled_edge = resample_edge(edge,num_pts_to_be_inserted)
#                 for i in range(len(resampled_edge)):
#                     resampled_pts.append(resampled_edge[i])
    
    if two_d_only:
        pca = PCA(n_components=2,)
        pts = np.array(apical_dendrite_pts)[:,0:2]
        transformed_pca = pca.fit_transform(X=pts)
    else:
        pca = PCA(n_components=3,)
        transformed_pca = pca.fit_transform(X=apical_dendrite_pts)
    #print(transformed_pca)
    return pca.components_

In [ ]:
def get_wm_intersection_pt(wm,soma_pt,pia_pt):
    wm_pt = []
    for j in range(500,6000,50):
        opposing_pt = Vectors().create_pt_along_vector_at_given_distance(-j,soma_pt,pia_pt)
        #print(opposing_pt)
        #Landmarks(pts=[opposing_pt]).write_landmarks(output_root+'oppsing_pt')
        wm_pt,dist = wm.get_vector_intersection_pt(soma_pt,opposing_pt,extrapolation_len=0)
        #Landmarks(pts=[opposing_pt]).write_landmarks(output_root+'wm_pt')
        #print(wm_pt)
        if len(wm_pt)>0:
            break
    return wm_pt

In [38]:
def get_cell_type_from_df(df,exp_name):
    for i in range(len(df)):
        if exp_name.startswith(df['file_id'][i]):
            return df['celltype_manual'][i]
    

# Create Pia and WM Surfaces

In [ ]:
@delayed
def create_surface(file,surface_resolution='100'):
    sg = AmiraSpatialGraph(filename=file)
    # flip if required
    #txmat = get_axis_flip_tx_mat(sg)
    #print(txmat)
    #sg.apply_transformation(txmat)
    #sg.write_spatial_graph(root_dst+os.path.basename(file))
    #sg = AmiraSpatialGraph(root_dst+os.path.basename(file))
    #sg_clipped = get_fist_x_z_planes_sg(sg,17)
    #sg_clipped.write_spatial_graph(root_dst+os.path.basename(file)+'_clipped.am')
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(100)))
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
    
    if pia.surface.GetNumberOfCells()>0 and wm.surface.GetNumberOfCells()>0:
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        
        os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(450)))
        pia_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        pia_450.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_450.vtk')
        
        wm_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        wm_450.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_450.vtk')
            
    else:
        # create a delunay surface as the traditional way has failed
        pia = Surface(pts=sg.pia.edge_pt_coords)
        wm = Surface(pts=sg.wm.edge_pt_coords)
        pia_hull = pia.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm_hull = wm.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        
        # clip at the bottom
        shutil.copyfile(src=file,dst=output_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        #wm.create_axis_field(pia,op_sg_name=output_surfaces+os.path.basename(file)[:-3]+'_axis_field.am',flip_normals=False,\
        #                              return_axes=True)

In [ ]:
fn_list_surfs = []
file_names = sorted(glob.glob(input_path_spatial_graphs+'*.am'))
for i in range(len(file_names)):
    #if os.path.basename(file_names[i])=='RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE.am':
    fn_list_surfs.append(create_surface(file_names[i]))

compute(fn_list_surfs)

# Generate Axis Field

In [ ]:
# Create axis with the smooth wm (voxel size 450) 
file_names = sorted(glob.glob(input_path_spatial_graphs+'*.am'))
for i in range(len(file_names)):
    file = file_names[i]
    #if os.path.basename(file)[:-3] == 'RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE':
    pia_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    wm_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_450.vtk')

    if wm_450.surface.GetNumberOfCells() > 0:
        wm_to_use = wm_450
    else:
        wm_to_use = wm_100
    
    flipped_axes = wm_to_use.create_axis_field(pia_100,op_sg_name=None,flip_normals=True,return_axes=True,\
                                            max_allowed_axis_length=5000,check_source_double_touch=True)
    axes = wm_to_use.create_axis_field(pia_100,op_sg_name=None,flip_normals=False,return_axes=True,\
                                    max_allowed_axis_length=5000,check_source_double_touch=True)
    sg = AmiraSpatialGraph(generic_graph=True)
    for edge in flipped_axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    for edge in axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    #sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')

    # trim the axis field as per the pia and wm surfaces of 100 micron voxel resolution
    selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia_100.get_axis_field_within_instersecting_surface_with_trimming\
                                (pia_100.surface,wm_100.surface,sg.graph_data.edge_list,vec_extension_offset=0.75)
    selected_axis_field_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')

        
        

# Stats of input neuron and surfaces

In [ ]:
cols = ['Exp_Name','Pia_Area','WM_Area','Pia_Volume','WM_Volume',\
        'Cortical_Thickness_Mean','Cortical_Thickness_Std','Cortical_Thickness_CoV',\
        'Cortical_Thickness_vS1_Mean','Cortical_Thickness_vS1_Std','Cortical_Thickness_vS1_CoV',]
df_main = pd.DataFrame(columns=cols)

for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(file)
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    pia_closed = pia.create_delunay_surface_3d(return_hull=True,make_cube=False)
    wm_closed = wm.create_delunay_surface_3d(return_hull=True,make_cube=False)
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    sg = AmiraSpatialGraph(file)
    vs1_sg = AmiraSpatialGraph(generic_graph=True)
    # create hull around s1 and get axis field for s1
    for pt in sg.barrels.all_rows_graphdata.edge_pt_coords:
        min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
        Vectors().get_nearest_axis_to_pt(pt,axis_field.graph_data.edge_list,axis_validation_distance=3000)
        vs1_sg.graph_data.add_edge(min_edge[0],min_edge[1])
    vs1_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_vS1_Axis_Field.am')
    
    # find vector lengths for whole surface and vs1 area
    vec_lens,bla1,bla2 = Vectors().get_vec_lengths(axis_field.graph_data.edge_list)
    vec_lens_vs1,bla,blee = Vectors().get_vec_lengths(vs1_sg.graph_data.edge_list)
        
    df = pd.DataFrame(columns=df_main.columns)
    df['Exp_Name'] = [os.path.basename(file)[:-3]]
    df['Pia_Area'] = [pia.get_surface_properties(prop='area')]
    df['WM_Area'] = [wm.get_surface_properties(prop='area')]
    df['Pia_Volume'] = [Surface(polydata=pia_closed).get_surface_properties(prop='volume')]
    df['WM_Volume'] = [Surface(polydata=wm_closed).get_surface_properties(prop='volume')]
    df['Cortical_Thickness_Mean'] = [np.array(vec_lens).mean(axis=0)]
    df['Cortical_Thickness_Std'] = [np.array(vec_lens).std(axis=0)]
    df['Cortical_Thickness_CoV'] = df['Cortical_Thickness_Std'] / df['Cortical_Thickness_Mean'] * 100
    df['Cortical_Thickness_vS1_Mean'] = [np.array(vec_lens_vs1).mean(axis=0)]
    df['Cortical_Thickness_vS1_Std'] = [np.array(vec_lens_vs1).std(axis=0)]
    df['Cortical_Thickness_vS1_CoV'] = df['Cortical_Thickness_vS1_Std'] / df['Cortical_Thickness_vS1_Mean'] * 100
        
    df_main = df_main.append(df)

df_main.to_csv(output_stats+'Surface_Area_and_Thickness.csv')
    

In [ ]:
cols = ['Exp_Name','Apical_Dendrite_Present','WM_Present_Below_Neuron',\
        'Neuron_Depth_from_Pia','Neuron_Height_from_WM','Neuron_Axis_Length']
df_main = pd.DataFrame(columns=cols)
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20160401_Cell_B_DONE':
    print(os.path.basename(file)[:-3])
    df = pd.DataFrame(columns=df_main.columns)
    df['Exp_Name'] = [os.path.basename(file)[:-3]]
    
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    sg = AmiraSpatialGraph(file)
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    Landmarks(pts=[soma_center]).write_landmarks(output_neuron_axis+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii')

    dendrite_pts = []    
    for pt in sg.neuron.dendrite.apical_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)
    for pt in sg.neuron.dendrite.basal_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)   
    dendrite_mean = np.array(dendrite_pts).mean(axis=0)

    # write dendrite for easy visualization
    sg.write_sub_spatial_graph(sg.neuron.dendrite.apical_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_apical_dendrite.am')
    sg.write_sub_spatial_graph(sg.neuron.dendrite.basal_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_basal_dendrite.am')

    # Avg Local Axis 
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field.graph_data.edge_list,[soma_center],800)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    nearby_axes_sg.write_spatial_graph(output_neuron_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*6000 + soma_center)
    pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)
    write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am')
    
    wm_proj_pt_neuron_using_local_avg_axis = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron_using_local_avg_axis)
    if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
        write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am')
    else:
        print('no wm intersection found using avg local axis for {}'.format(os.path.basename(file)))
    soma_depth_using_local_avg = Vectors().get_vec_length([soma_center,pia_proj_pt_neuron_using_local_avg_axis])
    # decide which method of depth measurement to use
    # using the apical dendrite axis works best for L5 and below neurons
    # where as local axis is good for L2/3
    print(soma_depth_using_local_avg)

    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
        df['Apical_Dendrite_Present'] = ['Yes']
        #df = pd.DataFrame(columns=df_main.columns)
        # Get apical dend axis using pca
        pca_components = get_pca_direction(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)
        #Landmarks(pts=resampled_pts).write_landmarks(output_root+'pts')
        end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
        #Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
        #print(end_pt)
        angle = Vectors().get_angle_between_vectors([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
                         [soma_center,end_pt+soma_center],ignore_opposite_direction=False)
        print(angle)
        pia_proj_pt_neuron = []
        if angle < 90:
            # pca direction is correct.. project to pia
            end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
            print(end_pt)
            Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
            pia_proj_pt_neuron,soma_depth  = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_pca_axis.am')
        else:
            # the pca is inverted, so straighten it
            #print('opposite')
            end_pt = Vectors().create_pt_along_vector_at_given_distance(-6000,[0,0,0],pca_components[0,:])
            pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_pca_axis.am')
        
        wm_proj_pt_neuron = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron)
        if len (wm_proj_pt_neuron) > 0:
            write_edge([soma_center,wm_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_pca_axis.am')
        else:
            print('no wm intersection found using pca for {}'.format(os.path.basename(file)))
    else:
        df['Apical_Dendrite_Present'] = ['No']
    # select the neron axis based on whether the depth of the neuron somata
    # for upper layer cells use avg axis, as they generally dont have enough apical dend
    # otherwise use dend pca based axis
    if soma_depth_using_local_avg < 500 or len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) == 0\
        or os.path.basename(file)[:-3] == 'RA20160401_Cell_B_DONE' or os.path.basename(file)[:-3]=='RA20180412_2_Cell_B.am_Segmented_corrected_DONE':
        # this is a inverted cell... so do not invert the axis.. lets use the avg axis for this one:
        df['Neuron_Depth_from_Pia'] = [distance.euclidean(soma_center,pia_proj_pt_neuron_using_local_avg_axis)]
        write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis.am')
        if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
            write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis.am')
            df['WM_Present_Below_Neuron'] = ['Yes']
            df['Neuron_Height_from_WM'] = [distance.euclidean(soma_center,wm_proj_pt_neuron_using_local_avg_axis)]
            df['Neuron_Axis_Length'] = df['Neuron_Depth_from_Pia'] + df['Neuron_Height_from_WM']
            
    else:
        df['Neuron_Depth_from_Pia'] = [distance.euclidean(soma_center,pia_proj_pt_neuron)]
        write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis.am')
        if len(wm_proj_pt_neuron) > 0:
            write_edge([soma_center,wm_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis.am')
            df['WM_Present_Below_Neuron'] = ['Yes']
            df['Neuron_Height_from_WM'] = [distance.euclidean(soma_center,wm_proj_pt_neuron)]
            df['Neuron_Axis_Length'] = df['Neuron_Depth_from_Pia'] + df['Neuron_Height_from_WM']
            
    df_main = df_main.append(df)
    
df_main.to_csv(output_stats+'Local_Neuron_Axis.csv')

# Register Using Barrels

In [ ]:
i = 0
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file) == 'RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE.am':
    print(i, os.path.basename(file))
    #break
    i = i+1
    sg = AmiraSpatialGraph(file,barrel_projections_present=False)

    if len(sg.barrels.all_rows_graphdata.edge_list) > 0:
        matching_barrels = MatchBarrels(ref_barrels.barrels,sg.barrels,use_barrels_alone=True)
        ref_s1_pts,s1_pts = matching_barrels.get_matching_barrel_centroids()
        landmarks = Landmarks(pts=s1_pts)
        icp,txmat = landmarks.align_landmarks(ref_s1_pts)
        aligner = Alignment()
        #print(txmat)
        #print(txmat)
        Alignment().transform_folders(output_root,output_alignment_using_barrels,os.path.basename(file)[:-3],\
                                icp=icp,txmat = txmat,)


# Register Using Surfaces

In [ ]:
# Using Pia and WM


In [ ]:
i = 0
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(i, os.path.basename(file))
    i = i + 1
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    
    pia_wm = pia
    pia_wm.append(wm.surface)

    aligner = Alignment(ref_pia_wm.surface,pia_wm.surface)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    aligner = Alignment()
    
    Alignment().transform_folders(output_root,output_alignment_using_surfaces,os.path.basename(file)[:-3],\
                            icp=icp,txmat = txmat,)
    
    

In [ ]:
# using only pias
output_path = output_alignment_using_surfaces+'/Pia_Only_Alignment/'
pathlib.Path(output_path).mkdir(exist_ok=True)

for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')

    aligner = Alignment(ref_pia.surface,pia.surface)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    aligner = Alignment()
    
    Alignment().transform_folders(output_root,output_path,os.path.basename(file)[:-3],\
                            icp=icp,txmat = txmat,)

In [ ]:
# using only wms
output_path = output_alignment_using_surfaces+'/WM_Only_Alignment/'
pathlib.Path(output_path).mkdir(exist_ok=True)

for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')

    aligner = Alignment(ref_wm.surface,wm.surface)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    aligner = Alignment()
    
    Alignment().transform_folders(output_root,output_path,os.path.basename(file)[:-3],\
                            icp=icp,txmat = txmat,)

In [ ]:
input_path = output_alignment_using_surfaces
output_path_nearest_axis = output_alignment_using_surfaces+'/Nearest_Axis/'
pathlib.Path(output_path_nearest_axis).mkdir(exist_ok=True)
avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'Avg_Axis_Field_50.am',generic_graph=True).graph_data.edge_list
nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
soma_pia_axes_sg = AmiraSpatialGraph(generic_graph=True)
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #print(i, os.path.basename(file))
    soma_center = Landmarks(input_path + '2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii').pts[0]
    neaerst_vec,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,avg_axis_field,axis_validation_distance=5000)
    nearest_axis_sg.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
    
    nearest_axis_sg_exp = AmiraSpatialGraph(generic_graph=True)
    nearest_axis_sg_exp.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
    nearest_axis_sg_exp.write_spatial_graph(output_path_nearest_axis+os.path.basename(file)[:-3]+'_nearest_axis.am')
    
    soma_pia_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                              +os.path.basename(file)[:-3]+'_soma_pia_axis.am',generic_graph=True).graph_data.edge_list[0]
    soma_pia_axes_sg.graph_data.add_edge(soma_pia_axis[0],soma_pia_axis[1])
nearest_axis_sg.write_spatial_graph(input_path+'Nearest_Axes.am')
soma_pia_axes_sg.write_spatial_graph(input_path+'Original_Soma_Pia_Axes.am')

# Refine Using Neuron Axis And Ref Axis

In [ ]:
def refine_registration_using_neuron_axis(input_path,output_path,preserve_rel_depth=True,use_nearest_axis=True):
    ''' Refine registration after the surface to surface registration is done, by aligning neuron axis with ref axis
        use_nearest_axis: whether to use nearest axis in the ref frame or to use the local avg axis
        preserve_rel_depth:whether to preserve the pia soma distance or the relative soma depth in the axis
    '''
    # read ref pia and wm 
    ref_pia = Surface(input_path_avg_rf+'Avg_Pia_50.vtk')
    ref_wm = Surface(input_path_avg_rf+'Avg_WM_50.vtk')
    
    output_path_nearest_axis = input_path+'/Nearest_Axis/'
    pathlib.Path(output_path_nearest_axis).mkdir(exist_ok=True)
    output_path_interpolated_ref_axis = input_path+'/Interpolated_Reference_Axis/'
    pathlib.Path(output_path_interpolated_ref_axis).mkdir(exist_ok=True)
    
    avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'Avg_Axis_Field_50.am',generic_graph=True).graph_data.edge_list
    nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        print(os.path.basename(file))
        original_axis_field = AmiraSpatialGraph(input_path+'1_Axis_Fields/'+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True).graph_data.edge_list

        soma_center = Landmarks(input_path + '2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii').pts[0]
        soma_pia_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                                  +os.path.basename(file)[:-3]+'_soma_pia_axis.am',generic_graph=True).graph_data.edge_list[0]
        
        if use_nearest_axis:
            neaerst_vec,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,avg_axis_field,axis_validation_distance=5000)
            nearest_axis_sg.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
            ref_vec_len = Vectors().get_vec_length(neaerst_vec) 
            nearest_axis_sg_exp = AmiraSpatialGraph(generic_graph=True)
            nearest_axis_sg_exp.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
            nearest_axis_sg_exp.write_spatial_graph(output_path_nearest_axis+os.path.basename(file)[:-3]+'_nearest_axis.am')
        else:
            # find the interpolated axis in the ref frame at this position 
            nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(avg_axis_field,[soma_center],800)
            nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
            unit_vecs = []
            for axis in nearby_axes:
                nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
                unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
            nearby_axes_sg.write_spatial_graph(output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
            avg_uv = np.array(unit_vecs).mean(axis=0)
            end_pt = (avg_uv*6000 + soma_center)
            #print(end_pt)
            wm_proj_pt_neuron_using_local_avg_axis,dist = ref_wm.get_vector_intersection_pt(end_pt,\
                                                        soma_center,extrapolation_len=10000)
            #print(wm_proj_pt_neuron_using_local_avg_axis)
            if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
                write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am')
                
                pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = ref_pia.get_vector_intersection_pt(wm_proj_pt_neuron_using_local_avg_axis,\
                                                                                            soma_center,extrapolation_len=10000)
                write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
                       output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am')
            
                neaerst_vec = [wm_proj_pt_neuron_using_local_avg_axis,pia_proj_pt_neuron_using_local_avg_axis]
            else:
                print('no wm intersection found using avg local axis for {}'.format(os.path.basename(file)))
            
            
        if preserve_rel_depth:
            # need to find the pia wm dist in the original neuron axis
            if os.path.exists(input_path+'2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am'):
                soma_wm_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                                          +os.path.basename(file)[:-3]+'_soma_wm_axis.am',generic_graph=True).graph_data.edge_list[0]
                original_vec_length = Vectors().get_vec_length(soma_pia_axis) + Vectors().get_vec_length(soma_wm_axis)
            else:
                # wm intersection not found.. try to find the nearest axis length in the original
                neaerst_vec_original,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,original_axis_field,axis_validation_distance=5000)
                original_vec_length = Vectors().get_vec_length(neaerst_vec_original) 


            # Get soma and pia pts on the ref axis based on the scaled soma pia depth
            len_ratio = Vectors().get_vec_length(neaerst_vec)/original_vec_length
            new_soma_pia_dist = len_ratio * Vectors().get_vec_length(soma_pia_axis) 
            soma_pt_on_ref_axis = Vectors().create_pt_along_vector_at_given_distance(new_soma_pia_dist,\
                                            neaerst_vec[1],neaerst_vec[0])
            pia_pt_on_ref_axis = Vectors().create_pt_along_vector_at_given_distance(Vectors().get_vec_length(soma_pia_axis),\
                                            soma_pt_on_ref_axis,neaerst_vec[1])

            landmarks = Landmarks(pts=soma_pia_axis)
            icp,txmat = landmarks.align_landmarks([soma_pt_on_ref_axis,pia_pt_on_ref_axis])
        else:
            # conserve the pia to soma dist only
            soma_pt_on_ref_axis = Vectors().create_pt_along_vector_at_given_distance(Vectors().get_vec_length(soma_pia_axis),\
                                            neaerst_vec[1],neaerst_vec[0])
            
            landmarks = Landmarks(pts=soma_pia_axis)
            icp,txmat = landmarks.align_landmarks([soma_pt_on_ref_axis,neaerst_vec[1]])
        
        #print(txmat)

        aligner2 = Alignment() 
        aligner2.transform_folders(input_path,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
        
        

In [ ]:
# register using 4 setups 
# preserving relative depth or pia soma dist
# using nearest axis in the ref frame or local avg axis 
refine_registration_using_neuron_axis(output_alignment_using_surfaces,output_alignment_using_neuron_axis_and_nearest_rel_depth,\
                                      use_nearest_axis=True,preserve_rel_depth=True)

refine_registration_using_neuron_axis(output_alignment_using_surfaces,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                                      use_nearest_axis=True,preserve_rel_depth=False)

refine_registration_using_neuron_axis(output_alignment_using_surfaces,output_alignment_using_neuron_axis_and_local_avg_rel_depth,\
                                      use_nearest_axis=False,preserve_rel_depth=True)

refine_registration_using_neuron_axis(output_alignment_using_surfaces,output_alignment_using_neuron_axis_and_local_Avg_pia_soma,\
                                      use_nearest_axis=False,preserve_rel_depth=False)

In [ ]:
for input_path in [output_alignment_using_neuron_axis_and_nearest_rel_depth,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                   output_alignment_using_neuron_axis_and_local_avg_rel_depth,output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    apical = AmiraSpatialGraph()
    basal = AmiraSpatialGraph()
    dend = AmiraSpatialGraph()
    #axon = AmiraSpatialGraph()
    neuron = AmiraSpatialGraph()
    #input_path = output_alignment_using_neuron_axis
    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        sg = AmiraSpatialGraph(input_path+'1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
        neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
        if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
            apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
        if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
            basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])

    neuron.write_spatial_graph(input_path+'All_Neurons_Registered.am')
    apical.write_spatial_graph(input_path+'All_Apical_Dendrites_Registered.am')
    basal.write_spatial_graph(input_path+'All_Basal_Dendrites_Registered.am')

# Create Cononical View

In [4]:
# align everything to coronal pia wm to create coronal view
aligner = Alignment(coronal_pia_wm.surface,ref_pia_wm.surface)
txmat = aligner.get_transformation_matrix()
icp = aligner.get_icp_transform()


In [7]:
aligner = Alignment()
output_canonical_rf = output_canonical+ '/Input_Ref_Frame/'
pathlib.Path(output_canonical_rf).mkdir(exist_ok=True)
Alignment().transform_folder(input_path_avg_rf,output_canonical_rf,\
                    icp=icp,txmat = txmat,)

#Alignment().transform_folders(output_root,output_canonical,exp_name=None,\
#                    icp=icp,txmat = txmat,)

In [49]:
Alignment().transform_folders(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'Manual_Cell_Type_Assignment/',output_canonical+'Manual_Cell_Type_Assignment/',\
                    icp=icp,txmat = txmat,)

# Sort Regiseted Neurons as per manual cell type assignment

In [50]:
# write individual neurons into their cell type folder
# Also write the cell layer as per registered depth

#input_path = output_alignment_using_neuron_axis + '1_Surfaces/'
cell_types_df = pd.read_excel(manual_cell_type_assignment,sheetname='celltype_MO')
#output_path_cell_type = output_alignment_using_neuron_axis+'/Manual_Cell_Type_Assignment/'
#pathlib.Path(output_path_cell_type).mkdir(exist_ok=True)

for input_path in [output_alignment_using_neuron_axis_and_nearest_rel_depth,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                   output_alignment_using_neuron_axis_and_local_avg_rel_depth,output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    
    output_path_cell_type = input_path+'/Manual_Cell_Type_Assignment/'
    pathlib.Path(output_path_cell_type).mkdir(exist_ok=True)

    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        #for i in range(len(cell_types_df)):
        exp_name = os.path.basename(file)[:-3]
        sg = AmiraSpatialGraph(input_path+'1_Surfaces/'+exp_name+'_barrels.am')
        cell_type = get_cell_type_from_df(cell_types_df,exp_name)
        print(exp_name,cell_type)
        
        outputpath = output_path_cell_type+'/{}/'.format(cell_type)
        pathlib.Path(outputpath).mkdir(exist_ok=True)
        apical = AmiraSpatialGraph()
        basal = AmiraSpatialGraph()
        dend = AmiraSpatialGraph()
        #axon = AmiraSpatialGraph()
        neuron = AmiraSpatialGraph()

        neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
        if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
            apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
            dend.graph_data = dend.combine_subgraphs([dend.graph_data,sg.neuron.dendrite.apical_dendrite])
        if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
            basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])
            dend.graph_data = dend.combine_subgraphs([dend.graph_data,sg.neuron.dendrite.basal_dendrite])

        neuron.write_spatial_graph(outputpath+exp_name+'_Neuron.am')
        apical.write_spatial_graph(outputpath+exp_name+'_Apical.am')
        basal.write_spatial_graph(outputpath+exp_name+'_Basal.am')
        dend.write_spatial_graph(outputpath+exp_name+'_Dend.am')
    

RA20130319_2_DONE L5tt
RA20130320_corrected_DONE L3
RA20130402_corrected_2_wm_countours_corrected_shifted_and_merged_DONE L6ct
RA20130404_1_DONE L5st
RA20130404_2_corrected_DONE L6cc
RA20130507_1_corrected_shifted_and_merged_DONE_feature_corrected L5tt
RA20130507_2_corrected_DONE L3
RA20130508_1_corrected_shifted_and_merged_DONE L5tt
RA20130513_corrected-extended_2corrected_shifted_and_merged_DONE L3
RA20130603_DONE L5tt
RA20130605_1_DONE L5st
RA20130606_DONE L2
RA20130607_1_DONE_feature_corrected_corrected inv
RA20130607_2_DONE L3
RA20130626_1_corrected_shifted_and_merged_DONE_feature_corrected L5tt
RA20130626_2_DONE L2
RA20130627_Cell_2_DONE L3
RA20130627_Cell_3_DONE L5tt
RA20130627_DONE L2
RA20130628_2_Cell_1_corrected_shifted_and_merged_DONE L2
RA20130628_2_Cell_2_corrected_shifted_and_merged_DONE L3
RA20130628_2_Cell_3_corrected_shifted_and_merged_DONE_feature_corrected_corrected L6cc
RA20130628_shifted_and_merged_DONE L5tt
RA20130701_2_DONE L3
RA20130722_2_corrected_shifted_and_m

RA20180109_2_Cluster_B_Cell_B_done.am_Segmented_corrected_DONE L3
RA20180109_2_Cluster_B_Cell_C_done.am_Segmented_corrected_DONE L3
RA20180109_2_Cluster_B_Cell_D_done.am_Segmented_corrected_DONE L2
RA20180109_2_Cluster_B_Cell_E_done.am_Segmented_corrected_DONE L4ss
RA20180411_final_Contours_merged.am_Segmented_corrected_shifted_and_merged_DONE L3
RA20180412_2_Cell_A.am_Segmented_corrected_DONE L3
RA20180412_2_Cell_B.am_Segmented_corrected_DONE ?
RA20180412_2_Cell_C.am_Segmented_corrected_DONE L2
RA20180412_2_Cell_D.am_Segmented_segmented_DONE L2
RA20180412_2_Cell_E.am_Segmented_corrected_shifted_and_merged_DONE L2
RA20130319_2_DONE L5tt
RA20130320_corrected_DONE L3
RA20130402_corrected_2_wm_countours_corrected_shifted_and_merged_DONE L6ct
RA20130404_1_DONE L5st
RA20130404_2_corrected_DONE L6cc
RA20130507_1_corrected_shifted_and_merged_DONE_feature_corrected L5tt
RA20130507_2_corrected_DONE L3
RA20130508_1_corrected_shifted_and_merged_DONE L5tt
RA20130513_corrected-extended_2corrected_s

RA20171215_1_.am_Segmented_corrected_DONE L6ct
RA20180108_1.am_Segmented_corrected_shifted_and_merged_DONE L2
RA20180108_2.am_Segmented_corrected_shifted_and_merged_DONE L2
RA20180109_1_Cell_A_done.am_Segmented_corrected_shifted_and_merged_DONE L2
RA20180109_1_Cell_B.am_Segmented_corrected_shifted_and_Merged_DONE L3
RA20180109_1_Cell_C_done.am_Segmented_corrected_DONE L2
RA20180109_2_Cluster_A_Cell_A_done.am_Segmented_corrected_DONE L2
RA20180109_2_Cluster_A_Cell_B_done.am_Segmented_corrected_shifted_and_merged_DONE L2
RA20180109_2_Cluster_A_Cell_C_done.am_Segmented_corrected_DONE L2
RA20180109_2_Cluster_B_Cell_A_done.am_Segmented_corrected_DONE L2
RA20180109_2_Cluster_B_Cell_B_done.am_Segmented_corrected_DONE L3
RA20180109_2_Cluster_B_Cell_C_done.am_Segmented_corrected_DONE L3
RA20180109_2_Cluster_B_Cell_D_done.am_Segmented_corrected_DONE L2
RA20180109_2_Cluster_B_Cell_E_done.am_Segmented_corrected_DONE L4ss
RA20180411_final_Contours_merged.am_Segmented_corrected_shifted_and_merged_DO

In [51]:
# combine all type specific neurons into each cell type class
#input_folder = output_alignment_using_neuron_axis+'/Manual_Cell_Type_Assignment/'
for input_folder in [output_alignment_using_neuron_axis_and_nearest_rel_depth,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                   output_alignment_using_neuron_axis_and_local_avg_rel_depth,output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    
    for folder in glob.glob(input_folder+'/Manual_Cell_Type_Assignment/*'):
        print(folder)
        
        # combine all neurons
        apical = AmiraSpatialGraph()
        basal = AmiraSpatialGraph()
        dend = AmiraSpatialGraph()
        neuron = AmiraSpatialGraph()
        for file in (glob.glob(folder+'/*_Apical.am')):
            sg_apical = AmiraSpatialGraph(file)
            apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg_apical.graph_data])
        for file in (glob.glob(folder+'/*_Basal.am')):
            sg_basal = AmiraSpatialGraph(file)
            basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg_basal.graph_data])
        for file in (glob.glob(folder+'/*_Dend.am')):
            sg_dend = AmiraSpatialGraph(file)
            dend.graph_data = dend.combine_subgraphs([dend.graph_data,sg_dend.graph_data])
        for file in (glob.glob(folder+'/*_Neuron.am')):
            sg_neuron = AmiraSpatialGraph(file)
            neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg_neuron.graph_data])

        neuron.write_spatial_graph(folder+'/All_Neuron_{}.am'.format(os.path.basename(folder)))
        apical.write_spatial_graph(folder+'/All_Apical_{}.am'.format(os.path.basename(folder)))
        basal.write_spatial_graph(folder+'/All_Basal_{}.am'.format(os.path.basename(folder)))
        dend.write_spatial_graph(folder+'/All_Dend_{}.am'.format(os.path.basename(folder)))
    
    

/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L5tt
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L3
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L6ct
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L5st
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L6cc
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L2
/nas1/Data_Mythreya/MotorCortexP

# Standardize and Write Galleries

In [106]:
def draw_horzontal_lines(a,b,op_filename):
    l1_line_v1 = AmiraSpatialGraph(generic_graph=True)
    l1_line_v1.graph_data.add_edge(a,b)
    l1_line_v1.write_spatial_graph(op_filename)

In [143]:
def draw_horzontal_lines_given_depth_profile(ref_len,layer_borders,layer_borders_std,layer_border_names,op_path):
    pia_line = AmiraSpatialGraph(generic_graph=True)
    pia_line.graph_data.add_edge([0,0,-0],[20000,0,-0])
    pia_line.write_spatial_graph(op_path+'Pia_Line_v2.am')
    i = 0
    for border in layer_borders:
        
        line = AmiraSpatialGraph(generic_graph=True)
        line_err_band = AmiraSpatialGraph(generic_graph=True)
        #line_ustd = AmiraSpatialGraph(generic_graph=True)
        #line_lstd = AmiraSpatialGraph(generic_graph=True)
        line.graph_data.add_edge([0,0,-(ref_len*border)],[20000,0,-(ref_len*border)])
        line_err_band.graph_data.add_edge([0,0,-(ref_len*border - ref_len*layer_borders_std[i] )],[20000,0,-(ref_len*border - ref_len*layer_borders_std[i])])
        line_err_band.graph_data.add_edge([0,0,-(ref_len*border + ref_len*layer_borders_std[i])],[20000,0,-(ref_len*border + ref_len*layer_borders_std[i])])
        line.write_spatial_graph( op_path + layer_border_names[i] + '_Line_v2.am' )
        #line_ustd.write_spatial_graph( op_path + layer_border_names[i] + '_Line_upper_band_v2.am' )
        #line_lstd.write_spatial_graph( op_path + layer_border_names[i] + '_Line_lower_band_v2.am' )
        
        # write the boundary error band as a vtk file
        Surface(pts=line_err_band.graph_data.edge_pt_coords).create_delunay_surface_3d(make_cube=True,output_filename = op_path + layer_border_names[i] + '_Error_Band.vtk' )
        
        i = i +1
    wm_line = AmiraSpatialGraph(generic_graph=True)
    wm_line.graph_data.add_edge([0,0,-ref_len],[20000,0,-ref_len])
    wm_line.write_spatial_graph(op_path+'WM_Line_v2.am')

In [144]:
# Find the central axis as the centroid of the registered somas
borders_folder = output_standardization+'/Central_Axis_And_Boders/'
pathlib.Path(borders_folder).mkdir(exist_ok=True)
registered_soma_centers = []
for file in sorted(glob.glob(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'/2_Local_Neuron_Axis/'+'*_Soma_Centroid.landmarksAscii')):
    registered_soma_centers.append(Landmarks(file).pts[0])
centroid = np.array(registered_soma_centers).mean(axis=0)
Landmarks(pts=[centroid]).write_landmarks(borders_folder+'somas_centroid.landmarksAscii')
Landmarks(pts=registered_soma_centers).write_landmarks(borders_folder+'registered_soma_centers.landmarksAscii')

# generate a central axis at the somas centroid in the avg rf
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_50.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_50.vtk')

avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'Avg_Axis_Field_50.am',generic_graph=True).graph_data.edge_list
nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(avg_axis_field,[centroid],800)
nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
unit_vecs = []
for axis in nearby_axes:
    nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
    unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
#nearby_axes_sg.write_spatial_graph(output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
avg_uv = np.array(unit_vecs).mean(axis=0)
end_pt = (avg_uv*6000 + soma_center)
#print(end_pt)
wm_proj_pt_neuron_using_local_avg_axis,dist = ref_wm.get_vector_intersection_pt(end_pt,\
                                            centroid,extrapolation_len=10000)
pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = ref_pia.get_vector_intersection_pt(wm_proj_pt_neuron_using_local_avg_axis,\
                                                                                centroid,extrapolation_len=10000)
central_axis_sg = AmiraSpatialGraph(generic_graph=True)
central_axis_sg.graph_data.add_edge(wm_proj_pt_neuron_using_local_avg_axis,pia_proj_pt_neuron_using_local_avg_axis)
central_axis_sg.write_spatial_graph(borders_folder+'central_axis.am')
central_axis = [wm_proj_pt_neuron_using_local_avg_axis,pia_proj_pt_neuron_using_local_avg_axis]

# make layer border landmarks
border_names = ['L1','L2_3','L5A','L5B','L6A']
borders = [0.06,0.24,0.48,0.67,0.88]
bordes_std = [0,0.10,0.10,0.085,0.12]
draw_horzontal_lines_given_depth_profile(Vectors().get_vec_length(central_axis),borders,bordes_std,border_names,borders_folder_vertical)


# read layer border csv
# lb_df = pd.read_csv(layer_borders_v1)
# l1_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*(abs(lb_df['L1_Per_Depth'][6])/100)
# l2_3_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L2/3_Per_Depth'][6])/100
# l5A_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L5A_Per_Depth'][6])/100
# l5B_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L5B_Per_Depth'][6])/100
# l6A_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L6A_Per_Depth'][6])/100

# l1_pt = Vectors().create_pt_along_vector_at_given_distance(l1_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l2_3_pt = Vectors().create_pt_along_vector_at_given_distance(l2_3_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l5A_pt = Vectors().create_pt_along_vector_at_given_distance(l5A_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l5B_pt = Vectors().create_pt_along_vector_at_given_distance(l5B_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l6A_pt = Vectors().create_pt_along_vector_at_given_distance(l6A_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)

# Landmarks(pts=[pia_proj_pt_neuron_using_local_avg_axis]).write_landmarks(borders_folder+'Pia_pt_v1.landmarkAscii')
# Landmarks(pts=[l1_pt]).write_landmarks(borders_folder+'L1_pt_v1.landmarkAscii')
# Landmarks(pts=[l2_3_pt]).write_landmarks(borders_folder+'L2_3_pt_v1.landmarkAscii')
# Landmarks(pts=[l5A_pt]).write_landmarks(borders_folder+'L5A_pt_v1.landmarkAscii')
# Landmarks(pts=[l5B_pt]).write_landmarks(borders_folder+'L5B_pt_v1.landmarkAscii')
# Landmarks(pts=[l6A_pt]).write_landmarks(borders_folder+'L6A_pt_v1.landmarkAscii')
# Landmarks(pts=[wm_proj_pt_neuron_using_local_avg_axis]).write_landmarks(borders_folder+'WM_pt_v1.landmarkAscii')

# # Verticalize all these things

# landmarks = Landmarks(pts=[central_axis[1],central_axis[0]])
# icp,txmat = landmarks.align_landmarks([[0,0,0],[0,0,-Vectors().get_vec_length(central_axis)]])
# borders_folder_vertical = borders_folder+'/Vertical/'
# pathlib.Path(borders_folder_vertical).mkdir(exist_ok=True)
# Alignment().transform_folder(borders_folder,borders_folder_vertical,txmat=txmat,icp=icp)

# # draw vertical border lines
# draw_horzontal_lines([0,0,-0],[20000,0,-0],borders_folder_vertical+'Pia_Line_v1.am')
# draw_horzontal_lines([0,0,-l1_dist],[20000,0,-l1_dist],borders_folder_vertical+'L1_Line_v1.am')
# draw_horzontal_lines([0,0,-l2_3_dist],[20000,0,-l2_3_dist],borders_folder_vertical+'L2_3_Line_v1.am')
# draw_horzontal_lines([0,0,-l5A_dist],[20000,0,-l5A_dist],borders_folder_vertical+'L5A_Line_v1.am')
# draw_horzontal_lines([0,0,-l5B_dist],[20000,0,-l5B_dist],borders_folder_vertical+'L5B_Line_v1.am')
# draw_horzontal_lines([0,0,-l6A_dist],[20000,0,-l6A_dist],borders_folder_vertical+'L6A_Line_v1.am')
# draw_horzontal_lines([0,0,-Vectors().get_vec_length(central_axis)],[20000,0,-Vectors().get_vec_length(central_axis)],borders_folder_vertical+'WM_Line_v1.am')


In [79]:
# read the cell type specific neurons, verticalize them and organize into galleries
standardized_no_scaling_neurons_path = output_standardization + '/Standardized_No_Scaling/' 
standardized_uniform_scaling_neurons_path = output_standardization  + '/Standardized_Uniform_Scaling/' 
standardized_layerwise_scaling_neurons_path = output_standardization  + '/Standardized_Layerwise_Scaling/' 
pathlib.Path(standardized_no_scaling_neurons_path).mkdir(exist_ok=True)
pathlib.Path(standardized_uniform_scaling_neurons_path).mkdir(exist_ok=True)
pathlib.Path(standardized_layerwise_scaling_neurons_path).mkdir(exist_ok=True)
central_axis = AmiraSpatialGraph(output_standardization+'central_axis.am',generic_graph=True).graph_data.edge_list[0]
for input_folder in [output_alignment_using_neuron_axis_and_local_avg_rel_depth,\
                     output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    
    #print(os.path.basename(input_folder[:-1]))
    standardized_no_scaling_neurons_path = output_standardization + '/Standardized_No_Scaling/' + \
                                                os.path.basename(input_folder[:-1]) + '/'
    standardized_uniform_scaling_neurons_path = output_standardization  + '/Standardized_Uniform_Scaling/' + \
                                                    os.path.basename(input_folder[:-1])+ '/'
    standardized_layerwise_scaling_neurons_path = output_standardization  + '/Standardized_Layerwise_Scaling/' + \
                                                    os.path.basename(input_folder[:-1])+ '/'
    pathlib.Path(standardized_no_scaling_neurons_path).mkdir(exist_ok=True)
    pathlib.Path(standardized_uniform_scaling_neurons_path).mkdir(exist_ok=True)
    pathlib.Path(standardized_layerwise_scaling_neurons_path).mkdir(exist_ok=True)
    
    for folder in sorted(glob.glob(input_folder+'/Manual_Cell_Type_Assignment/*')):
        print(folder)
        gallery_no_scaling_sg = AmiraSpatialGraph()
        gallery_uniform_scaling_sg = AmiraSpatialGraph()
        gallery_layerwise_scaling_sg = AmiraSpatialGraph()
        gallery_all_sg = AmiraSpatialGraph()
        i = 500
        
        for file in sorted(glob.glob(folder+'/*_Neuron.am')):
            #i = i + 1
            exp_name = os.path.basename(file)[:-10]
            # first verticalize for gallery view
            sg_neuron = AmiraSpatialGraph(file)
            sg_dend = AmiraSpatialGraph(file[:-10]+'_Dend.am')
            # read the soma pia axis and soma center
            if input_folder.find('Nearest')>0:
                soma_center = Landmarks(input_folder+'/2_Local_Neuron_Axis/'+exp_name+\
                                        '_Soma_Centroid.landmarksAscii').pts[0]
            
                neuron_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'/Nearest_Axis/'+exp_name+'_nearest_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
            else:
                soma_center = Landmarks(input_folder+'/2_Local_Neuron_Axis/'+exp_name+\
                                        '_Soma_Centroid.landmarksAscii').pts[0]
            
                soma_pia_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'/Interpolated_Reference_Axis/'+\
                                                exp_name+'_soma_pia_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
                soma_wm_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'/Interpolated_Reference_Axis/'+\
                                                exp_name+'_soma_wm_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
                neuron_axis = [soma_wm_axis[1],soma_pia_axis[1]]
            
            soma_pia_axis = [soma_center, neuron_axis[1]]
            
            # align neuron to the negative z axis
            rot_mat = align_a_to_b(soma_pia_axis,[[0,0,0],[0,0,1]])
            translation = neuron_axis[1]

            sg_neuron.apply_rotation(rot_mat,translation=translation)
            sg_neuron.write_spatial_graph(standardized_no_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            sg_dend.apply_rotation(rot_mat,translation=translation)
            sg_dend.write_spatial_graph(standardized_no_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            
            # space them so as to visualize better
            sg_dend_tr = AmiraSpatialGraph(standardized_no_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron_tr = AmiraSpatialGraph(standardized_no_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_tr.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i,0,0])
            i = i + 400
            gallery_no_scaling_sg.graph_data = gallery_no_scaling_sg.combine_subgraphs([gallery_no_scaling_sg.graph_data,\
                                                                                    sg_dend_tr.graph_data])
            
            # scale uniformely to the central axis
            scaling_factor = Vectors().get_vec_length(central_axis) / Vectors().get_vec_length(neuron_axis)
            sg_dend.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_neuron.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_dend.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i,0,0])
            i = i + 600
            gallery_uniform_scaling_sg.graph_data = gallery_uniform_scaling_sg.combine_subgraphs([gallery_uniform_scaling_sg.graph_data,\
                                                                                    sg_dend_scaling.graph_data])
            
            
            # scale uniformely to the central axis
            scaling_factor = Vectors().get_vec_length(central_axis) / Vectors().get_vec_length(neuron_axis)
            sg_dend.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_neuron.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_dend.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i,0,0])
            i = i + 600
            gallery_uniform_scaling_sg.graph_data = gallery_uniform_scaling_sg.combine_subgraphs([gallery_uniform_scaling_sg.graph_data,\
                                                                                    sg_dend_scaling.graph_data])
            
        
        gallery_no_scaling_sg.write_spatial_graph(standardized_no_scaling_neurons_path+os.path.basename(folder)+'_No_Scaling_Gallery.am')
        gallery_uniform_scaling_sg.write_spatial_graph(standardized_uniform_scaling_neurons_path+os.path.basename(folder)+'_Uniform_Scaling_Gallery.am')
        gallery_uniform_scaling_sg.write_spatial_graph(standardized_uniform_scaling_neurons_path+os.path.basename(folder)+'_Uniform_Scaling_Gallery.am')
        

/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/?
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L2
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L3
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L4ss
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L5st
/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved//Manual_Cell_Type_Assignment/L5tt

In [33]:
# Find soma pia depth
input_sp_path = output_standardization + 'Standardized_No_Scaling/' + os.path.basename(output_alignment_using_neuron_axis_and_local_Avg_pia_soma[:-1])
#len(glob.glob(input_sp_path+'/*_Neuron_*.am'))
soma_depths_csv = pd.DataFrame(columns=['Exp_Name','Soma_Depth_From_Pia'])
for file in sorted(glob.glob(input_sp_path+'/*_Neuron_*.am')):
    sg = AmiraSpatialGraph(file)
    soma_centroid = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    Landmarks(pts=[soma_centroid]).write_landmarks(input_sp_path+'/'+os.path.basename(file)+'_Soma_Centroid.landmarksAscii')
    df=pd.DataFrame()
    df['Exp_Name'] = [os.path.basename(file)[:-14]]
    df['Soma_Depth_From_Pia'] = soma_centroid[2]
    soma_depths_csv = soma_depths_csv.append(df)
    #print(os.path.basename(file),soma_centroid,)
    
soma_depths_csv.to_csv(input_sp_path+'/Soma_Depth_From_Pia.csv')

In [34]:
input_sp_path

'/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Outputs/5_Standardized_Neurons/Standardized_No_Scaling/4_Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved'

# Create Avg Pia WM

In [ ]:
# combine all aligned pia wms
aligned_pias = Surface()
aligned_wms = Surface()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(os.path.basename(file)[:-3])
    pia = Surface(output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment/1_Surfaces/'+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_root+'3_Aligned_By_Surfaces/WM_Only_Alignment/1_Surfaces/'+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    aligned_pias.append(pia.surface)
    aligned_wms.append(wm.surface)
    
aligned_pias.write_surface_mesh(output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment/Aligned_Pias.vtk')
aligned_wms.write_surface_mesh(output_root+'3_Aligned_By_Surfaces/WM_Only_Alignment/Aligned_WMs.vtk')

In [ ]:
#aligned_pias = Surface(output_root+'3_Aligned_By_Surfaces/Aligned_Pias.vtk')
#aligned_wms = Surface(output_root+'3_Aligned_By_Surfaces/Aligned_WMs.vtk')
pia_wms = Surface()
pia_wms.append(aligned_pias.surface)
pia_wms.append(aligned_wms.surface)

centroid = pia_wms.get_center_of_mass()
ray_origin = [centroid[0],centroid[1],pia_wms.surface.GetBounds()[5]-1000]

pia_pts = aligned_pias.get_ray_cast_surface_pts_avg(center=ray_origin,theta_res=60,phi_res=60,\
                                                     start_phi=0,end_phi=360,start_theta=0,end_theta=360,validation_nr_pts=2)

Landmarks(pts=pia_pts).write_landmarks(output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment/Avg_Pia.landmarksAscii')

# wm_pts = aligned_wms.get_ray_cast_surface_pts_avg(center=ray_origin,theta_res=60,phi_res=60,\
#                                                  start_phi=0,end_phi=360,start_theta=0,end_theta=360,validation_nr_pts=1)

# Landmarks(pts=wm_pts).write_landmarks(output_root+'3_Aligned_By_Surfaces/WM_Only_Alignment/Avg_WM.landmarksAscii')

In [ ]:
wm_pts = aligned_wms.get_ray_cast_surface_pts_avg(center=ray_origin,theta_res=60,phi_res=60,\
                                                 start_phi=0,end_phi=360,start_theta=0,end_theta=360,validation_nr_pts=2)

Landmarks(pts=wm_pts).write_landmarks(output_root+'3_Aligned_By_Surfaces/WM_Only_Alignment/Avg_WM.landmarksAscii')

In [ ]:
pia_pts = Landmarks(filename=output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment/Avg_Pia.landmarksAscii')
pia_surf = Surface(pts=pia_pts.pts)
bounds = pia_surf.surface.GetBounds()
z_span = bounds[5] - bounds[4]
offset = z_span - 5000

recon = vtk.vtkSurfaceReconstructionFilter()
recon.SetSampleSpacing(100)
recon.SetNeighborhoodSize(100)
recon.SetInputData(Surface(pts=pia_pts.pts).surface)
recon.Update()

#     cont = vtk.vtkContourFilter()
#     cont.SetInputData(recon.GetOutput())
#     cont.SetValue(0,0.0)
#     cont.Update()

mc = vtk.vtkMarchingContourFilter()
mc.SetInputData(recon.GetOutput())
#mc.SetNumberOfContours(0)
mc.SetValue(0,0)
mc.Update()

rev = vtk.vtkReverseSense()
rev.SetInputData(mc.GetOutput())
rev.ReverseCellsOn()
rev.ReverseNormalsOn()
rev.Update()

clipped_surf = Surface(polydata=rev.GetOutput()).clip_surface_at_given_z(z_offset=-offset,z_limit=None)
#divided_surf = self.divide_surface(1,clipped_surf)

cleaner = vtk.vtkCleanPolyData()
cleaner.SetInputData(clipped_surf)
cleaner.Update()

smoothed_surf = Surface().smooth_surface(cleaner.GetOutput())

Surface(polydata=smoothed_surf).write_surface_mesh(output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment/Avg_Pia_50.vtk')


In [ ]:
# clean up the avg pts based on density ?
#for pt in pia_pts:
    # how far is it from the nearest pt
dm = distance_matrix(pia_pts.pts,pia_pts.pts)
dm.shape

In [ ]:
pia_pts = Landmarks(filename=output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment/Avg_Pia.landmarksAscii')
pia_surf = Surface(pts=pia_pts.pts)
bounds = pia_surf.surface.GetBounds()
z_span = bounds[5] - bounds[4]
offset = z_span - 5000
pia_surf.create_surface_from_unorganized_points(pia_pts.pts,sample_resolution=200,\
                                                                               clip_offset=-offset,z_limit=None,\
                                        output_filename=output_root+'3_Aligned_By_Surfaces/Pia_Only_Alignment'+'/Avg_Pia.vtk')

wm_pts = Landmarks(filename=output_root+'3_Aligned_By_Surfaces/WM_Only_Alignment/Avg_WM.landmarksAscii')
wm_surf = Surface(pts=wm_pts.pts)
wm_surf.create_surface_from_unorganized_points(wm_pts.pts,sample_resolution=200,\
                                                                               clip_offset=-offset,z_limit=None,\
                                        output_filename=output_root+'3_Aligned_By_Surfaces/WM_Only_Alignment'+'/Avg_WM.vtk')


# Final Registration

In [ ]:
def get_central_axis(center, uv, pia, wm, invert_direction=False):
    if invert_direction:
        pia_intersection_pt,dist= pia.get_vector_intersection_pt(center,center+uv,extrapolation_len=-10000)
    else:
        pia_intersection_pt,dist= pia.get_vector_intersection_pt(center,center+uv,extrapolation_len=10000)
    
    return [center,pia_intersection_pt]

In [ ]:
def get_available_barrel_central_axis(sg,pia,wm,vs1_sg):
    ''' Get barrel column in the following order D2, or center of vs1'''
    if len(sg.barrels.d_row.two.top_barrel_centroid)>0 and \
            len(sg.barrels.d_row.two.bottom_barrel_centroid)>0 and \
            sg.barrels.d_row.two.top_barrel_centroid[2] != sg.barrels.d_row.two.bottom_barrel_centroid[2]:
            
        center =  np.array(sg.barrels.d_row.two.contours.edge_pt_coords).mean(axis=0)
        uv = Vectors().get_unit_vec(sg.barrels.d_row.two.bottom_barrel_centroid,sg.barrels.d_row.two.top_barrel_centroid)
        central_axis = get_central_axis(center, uv, pia, wm,invert_direction=True)
    else:
        # use vs1 axis field to get the central axis
        center = np.array(sg.barrels.all_rows_graphdata.edge_pt_coords).mean(axis=0)
        uvs = []
        for edge in vs1_sg.graph_data.edge_list:
            uvs.append(Vectors().get_unit_vec(edge[0],edge[1]))
        mean_uv = np.array(uvs).mean(axis=0)
        central_axis = get_central_axis(center,mean_uv, pia, wm)
        
    return central_axis

## Align To Central Axis

In [ ]:
# 1) If available align using D2/D1 column to center it; otherwise use the center of mass and nearest axis. 
#    Then find the 2D pca and rotate. 
# 2) Then find the XY angle from center which convers the neuron
# 3) Clip the Putative vM1 surface containing the neuron
# 4) Align this vm1 with avg vm1
i = 0
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    i = i + 1
    print(i, os.path.basename(file))
    sg = AmiraSpatialGraph(file,barrel_projections_present=False)
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    
    output_path = output_alignment_final+'Aligned_To_Central_Axis/'
    pathlib.Path((output_path)).mkdir(exist_ok=True)
    
    # 1) If available align using D2/D1 column to center it; otherwise use the center of mass and nearest axis. 
    #    Then find the 2D pca and rotate. 
    if len(sg.barrels.all_rows_graphdata.edge_list) > 0:
        # read the vS1 axis field
        #if os.path.exists(output_axis_field+os.path.basename(file)[0:-3]+'_vS1_Axis_Field.am')
        vs1_axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_vS1_Axis_Field.am',\
                                           generic_graph=True)
        # align based on the central axis
        central_axis = get_available_barrel_central_axis(sg,pia,wm,vs1_axis_field)
        
        print('Using Barrels', central_axis )
        
        #pca_components = get_pca_direction(sg.graph_data.edge_pt_coords,two_d_only=True)
    else:
        # no barrels.. so get the central axis using local avg axis
        center = pia.get_center_of_mass()
        axis_field_sg = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am' ,generic_graph=True)
        nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field_sg.graph_data.edge_list,[center],1000,\
                                                                     axis_validation_distance=3000)
        nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
        unit_vecs = []
        for axis in nearby_axes:
            nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
            unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
        nearby_axes_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_vS1_Axis_Field.am')
        avg_uv = np.array(unit_vecs).mean(axis=0)
        
        central_axis = get_central_axis(center,avg_uv,pia,wm)
        print('Using surface' , central_axis)
        
#     central_axis_sg = AmiraSpatialGraph(generic_graph=True)
#     central_axis_sg.graph_data.add_edge(central_axis[0],central_axis[1])
#     central_axis_sg.write_spatial_graph(output_path+os.path.basename(file)[:-3]+'_Central_Axis.am')
    
    pts = []
    ref_pts = []
    pts.append(central_axis[0])
    pts.append(central_axis[1])
    ref_pts.append([0,0,0])
    ref_pts.append([0,0,-distance.euclidean(central_axis[0],central_axis[1])])
    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
    
    
    aligner = Alignment() 
    aligner.transform_folders(output_root,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
        
    

## Align as per Max Y

In [ ]:
i = 0
cols = ['Exp','Distance_of_Soma_From_Center','Distance_of_Soma_From_Center_XY','Angle_XY_of_Soma_From_Center','Angle_Z_of_Neuron_Axis_From_Center']
df_main = pd.DataFrame(columns=cols)
for file in sorted(glob.glob(output_alignment_final+'Aligned_To_Central_Axis/1_Surfaces/'+'*_barrels.am')):
    i = i + 1
    print(i, os.path.basename(file))
    sg = AmiraSpatialGraph(file,barrel_projections_present=False)
    
    #pca_components = get_pca_direction(sg.graph_data.edge_pt_coords,two_d_only=True)
    #txmat = [pca_components[1][0],pca_components[1][1],0,0,pca_components[0][0],pca_components[0][1],0,0, 0,0,1,0, 0,0,0,1]
#     surf = Surface(pts=sg.graph_data.edge_pt_coords)
#     print(surf.surface.GetBounds())
    maxy_pt = sg.graph_data.edge_pt_coords[np.argmin(np.array(sg.graph_data.edge_pt_coords)[:,1])]
    #print(maxy_pt)
    
    pts = []
    ref_pts = []
    pts.append([0,0,0])
    pts.append([maxy_pt[0],maxy_pt[1],0])
    ref_pts.append([0,0,0])
    dist = -distance.euclidean([0,0,0],[maxy_pt[0],maxy_pt[1],0])
    ref_pts.append([0,dist,0])
    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
    
    output_path = output_alignment_final+'Aligned_To_MaxY/'
    pathlib.Path((output_path)).mkdir(exist_ok=True)
    aligner = Alignment() 
    aligner.transform_folders(output_alignment_final+'Aligned_To_Central_Axis/',\
                              output_path,os.path.basename(file)[:-10],txmat=txmat,icp=icp)
    
    soma_center = Landmarks(output_neuron_axis+os.path.basename(file)[:-10]+'Soma_Centroid.landmarksAscii').pts[0]
    neuron_axis = AmiraSpatialGraph(output_neuron_axis+os.path.basename(file)[:-10]+'soma_pia_axis.am',generic_graph=True)
   

### Get Neuron's Position

In [ ]:
cols = ['Exp','Soma_Center_X','Soma_Center_Y','Soma_Center_Z','Distance_of_Soma_From_Center_3D',\
        'Angle_of_Soma_From_X_Axis','Angle_of_Soma_From_Z_Axis',\
        'Distance_To_Nearest_Axis_XY',\
        'Cortical_Thickness_As_Per_Neuron_Axis','Cortical_Thickness_As_Per_Avg_Local_Axis',\
        'Cortical_Thickness_As_Per_Nearest_Axis_in_Avg_Surface',\
        'Thickness_Difference_From_Neuron_Axis',\
        'Thickness_Difference_From_Neuron_Axis_Local_Avg',\
        'Angle_Between_Neuron_Axis_And_Local_Avg_Axis_Original',\
        'Angle_Between_Neuron_Axis_And_Nearest_Axis'
       ]
df_main = pd.DataFrame(columns=cols)
i = 0
nearest_axis_path = output_alignment_final+'Aligned_To_MaxY/'+'Nearest_Axis/'
pathlib.Path((nearest_axis_path)).mkdir(exist_ok=True)
nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'Avg_Axis_Field_50.am',generic_graph=True).graph_data.edge_list

for file in sorted(glob.glob(output_alignment_final+'Aligned_To_Central_Axis/1_Surfaces/'+'*_barrels.am')):
    i = i + 1
    print(i, os.path.basename(file))
    sg = AmiraSpatialGraph(file,barrel_projections_present=False)
    
    soma_center = Landmarks(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+os.path.basename(file)[:-10]+'Soma_Centroid.landmarksAscii').pts[0]
    neuron_soma_pia_axis = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+\
                                             os.path.basename(file)[:-10]+'soma_pia_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    neuron_soma_pia_axis_local_avg = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+\
                                             os.path.basename(file)[:-10]+'soma_pia_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    
    
    df = pd.DataFrame(columns=cols)
    df['Exp'] = [os.path.basename(file)[:-11]]
    df['Soma_Center_X'] = [soma_center[0]]
    df['Soma_Center_Y'] = [soma_center[1]]
    df['Soma_Center_Z'] = [soma_center[2]]
    df['Distance_of_Soma_From_Center_3D'] = [distance.euclidean([0,0,0],soma_center)]
    df['Angle_of_Soma_From_X_Axis'] = Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]], \
                                                                           [[0,0,0],[soma_center[0],soma_center[1],0]],\
                                                                          ignore_opposite_direction=True)
    df['Angle_of_Soma_From_Z_Axis'] = Vectors().get_angle_between_vectors([[0,0,0],[0,0,-1]], \
                                                                           [[0,0,0],[0,soma_center[1],soma_center[2]]],\
                                                                          ignore_opposite_direction=True)
    
    neaerst_vec,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,avg_axis_field,axis_validation_distance=5000)
    nearest_axis_sg.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
    
    
    df['Distance_To_Nearest_Axis_XY'] = distance.euclidean([neaerst_vec[1][0],neaerst_vec[1][1],0],\
                                                           [soma_center[0],soma_center[1],0])
    
    df['Cortical_Thickness_As_Per_Nearest_Axis_in_Avg_Surface'] = Vectors().get_vec_length(neaerst_vec)
        
    if os.path.exists(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+os.path.basename(file)[:-10]+\
                      'soma_wm_axis.am'):
        neuron_soma_wm_axis = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+\
                                                os.path.basename(file)[:-10]+'soma_wm_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
        
        df['Cortical_Thickness_As_Per_Neuron_Axis'] = Vectors().get_vec_length(neuron_soma_pia_axis)+\
                                                      Vectors().get_vec_length(neuron_soma_wm_axis)
        
        df['Thickness_Difference_From_Neuron_Axis'] = df['Cortical_Thickness_As_Per_Nearest_Axis_in_Avg_Surface'] - \
                                                        df['Cortical_Thickness_As_Per_Neuron_Axis']
        
    if os.path.exists(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+os.path.basename(file)[:-10]+\
                      'soma_wm_axis_local_avg.am'):
        neuron_soma_wm_axis_local_avg = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/2_Local_Neuron_Axis/'+\
                                                os.path.basename(file)[:-10]+'soma_wm_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
        df['Cortical_Thickness_As_Per_Avg_Local_Axis'] = Vectors().get_vec_length(neuron_soma_pia_axis_local_avg)+\
                                                         Vectors().get_vec_length(neuron_soma_wm_axis_local_avg)
        df['Thickness_Difference_From_Neuron_Axis_Local_Avg'] = df['Cortical_Thickness_As_Per_Nearest_Axis_in_Avg_Surface'] \
                                                                -  df['Cortical_Thickness_As_Per_Avg_Local_Axis']
    df['Angle_Between_Neuron_Axis_And_Local_Avg_Axis_Original'] = Vectors().get_angle_between_vectors(neuron_soma_pia_axis,\
                                                                                                      neuron_soma_pia_axis_local_avg,\
                                                                                                      ignore_opposite_direction=True)
    df['Angle_Between_Neuron_Axis_And_Nearest_Axis'] = Vectors().get_angle_between_vectors(neuron_soma_pia_axis,\
                                                                                           neaerst_vec,\
                                                                                           ignore_opposite_direction=True)
    df_main = df_main.append(df)
    
df_main.to_csv(output_root+'Neuron_Position_and_Angles.csv')
nearest_axis_sg.write_spatial_graph(nearest_axis_path+'Nearest_axes.am')

### Write Neurons for visulization

In [ ]:
apical = AmiraSpatialGraph()
basal = AmiraSpatialGraph()
dend = AmiraSpatialGraph()
#axon = AmiraSpatialGraph()
neuron = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    sg = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
        apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
    if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
        basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])
    
neuron.write_spatial_graph(output_alignment_final+'Aligned_To_MaxY'+'/All_Neurons.am')
apical.write_spatial_graph(output_alignment_final+'Aligned_To_MaxY'+'/All_Apical_Dendrites.am')
basal.write_spatial_graph(output_alignment_final+'Aligned_To_MaxY'+'/All_Basal_Dendrites.am')

## Clip Pia and WM Surfaces





In [ ]:
def clean_pdata(pdata):
    cleaner = vtk.vtkCleanPolyData()
    cleaner.SetInputData(pdata)
    cleaner.Update()
    
    return cleaner.GetOutput()

In [ ]:
output_path = output_alignment_final+'/Clipped_Surfaces/'
pathlib.Path((output_path)).mkdir(exist_ok=True)
cols = ['Exp_Name','vM1_Thickness_Mean','vM1_Thickness_Std']
df_main = pd.DataFrame(columns=cols)
# Clip between 55 degree and 95 degree in XY plane
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    pia = Surface(output_alignment_final+'Aligned_To_MaxY/1_Surfaces/'+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_alignment_final+'Aligned_To_MaxY/1_Surfaces/'+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    axis_field = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/1_Axis_Fields/'+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    # Get 3 pts for each angle 
    rotation_mat_right = (Vectors().get_rotation_matrix_around_given_axis(theta=-55,axis_of_rotation='z'))
    rotation_mat_left = (Vectors().get_rotation_matrix_around_given_axis(theta=-95,axis_of_rotation='z'))
    
    right_plane_pt = np.array(rotation_mat_right*[1,0,0])[:,0]
    left_plane_pt = np.array(rotation_mat_left*[1,0,0])[:,0]
    
    plane_right,tri_right = Surface().get_cutting_plane([0,0,0],[0,0,1],right_plane_pt)
    plane_left,tri_left = Surface().get_cutting_plane([0,0,0],[0,0,1],left_plane_pt)
    
    pia1,wm1 = pia.clip_surfaces(pia.surface,wm.surface,plane_right,get_left=False)
    pia2,wm2 = pia.clip_surfaces(pia1,wm1,plane_left,get_left=True)
    
    cleaned_pia = clean_pdata(pia2)
    cleaned_wm = clean_pdata(wm2)
    
    Surface(polydata=cleaned_pia).write_surface_mesh(output_alignment_final+'/Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_pia.vtk')
    Surface(polydata=cleaned_wm).write_surface_mesh(output_alignment_final+'/Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_WM.vtk')
    
    # get axis field within this
    selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
                                (pia2,wm2,axis_field.graph_data.edge_list,vec_extension_offset=0.5)
    selected_axis_field_sg.write_spatial_graph(output_alignment_final+'/Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_vM1_Axis_Field.am')
    
    # find lengths within this
    vec_lengths,a,b = Vectors().get_vec_lengths(selected_axis_field_sg.graph_data.edge_list)
    
    df = pd.DataFrame(columns=cols)
    df['Exp_Name'] = [os.path.basename(file)[:-3]]
    df['vM1_Thickness_Mean'] = np.array(vec_lengths).mean()
    df['vM1_Thickness_Std'] = np.array(vec_lengths).std()
    
    df_main = df_main.append(df)
    
    
df_main.to_csv(output_alignment_final+'/Clipped_Surfaces/'+'vM1_Thickness.csv')

In [ ]:
# Clip the avg surface
ref_pia = Surface(input_path_avg_rf+'/Avg_Pia_50.vtk')
ref_wm = Surface(input_path_avg_rf+'/Avg_WM_50.vtk')
ref_axis_field_sg = AmiraSpatialGraph(input_path_avg_rf+'Avg_Axis_Field_50.am',generic_graph=True)
rotation_mat_right = (Vectors().get_rotation_matrix_around_given_axis(theta=-55,axis_of_rotation='z'))
rotation_mat_left = (Vectors().get_rotation_matrix_around_given_axis(theta=-95,axis_of_rotation='z'))

right_plane_pt = np.array(rotation_mat_right*[1,0,0])[:,0]
left_plane_pt = np.array(rotation_mat_left*[1,0,0])[:,0]

plane_right,tri_right = Surface().get_cutting_plane([0,0,0],[0,0,1],right_plane_pt)
plane_left,tri_left = Surface().get_cutting_plane([0,0,0],[0,0,1],left_plane_pt)

pia1,wm1 = ref_pia.clip_surfaces(ref_pia.surface,ref_wm.surface,plane_right,get_left=False)
pia2,wm2 = ref_wm.clip_surfaces(pia1,wm1,plane_left,get_left=True)

cleaned_pia = clean_pdata(pia2)
cleaned_wm = clean_pdata(wm2)

Surface(polydata=cleaned_pia).write_surface_mesh(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Pia.vtk')
Surface(polydata=cleaned_wm).write_surface_mesh(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_WM.vtk')


# get axis field within this
selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
                            (pia2,wm2,ref_axis_field_sg.graph_data.edge_list,vec_extension_offset=0.5)
selected_axis_field_sg.write_spatial_graph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am')


In [ ]:
# compare the cortical thickness between Rajeev and Rabies vM1s
avg_vm1_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',generic_graph=True)
avg_vm1_lengths,a,b = Vectors().get_vec_lengths(avg_vm1_sg.graph_data.edge_list)
np.array(avg_vm1_lengths).mean(axis=0),np.array(avg_vm1_lengths).std(axis=0),np.array(avg_vm1_lengths).max(),np.array(avg_vm1_lengths).min()

In [ ]:
df = pd.read_csv(output_alignment_final+'/Clipped_Surfaces/'+'vM1_Thickness.csv')
df.mean(),df.min(),df.max()

## Align Clipped Avg with Clipped Surfaces

In [ ]:
def clean_axis_field(vm1_axis_field):
    clean_field_sg = AmiraSpatialGraph(generic_graph=True)
    cleaned_axis_field = []
    for edge in vm1_axis_field:
        if edge[0][2] > edge[1][2]:
            cleaned_axis_field.append(edge)
            clean_field_sg.graph_data.add_edge(edge[0],edge[1])
    #clean_field_sg.write_spatial_graph(output_path+'clean.am')
    return clean_field_sg

In [ ]:
def get_sg_from_edge_list(edge_list):
    sg = AmiraSpatialGraph(generic_graph=True)
    cleaned_axis_field = []
    for edge in edge_list:
        sg.graph_data.add_edge(edge[0],edge[1])
    #clean_field_sg.write_spatial_graph(output_path+'clean.am')
    return sg

In [ ]:
def get_number_of_axes_from_field(axis_field,number_of_axes_needed,neuron_axis=[]):
    axis_filed_len = len(axis_field)
    offset = int(axis_filed_len/number_of_axes_needed)
    return_list = []
    if len(neuron_axis) > 0:
        if neuron_axis is not None:
            for i in range(len(axis_field)):
                return_list.append(axis_field[i])
                i = i + offset
                if len(return_list) == number_of_axes_needed:
                    return return_list
        
    else:
        
        for i in range(len(axis_field)):
            return_list.append(axis_field[i])
            i = i + offset
            if len(return_list) == number_of_axes_needed:
                return return_list

### Using Clipped Surfaces

In [ ]:
# avg_vm1_axis_field_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',\
#                                        generic_graph=True)
# avg_vm1_axis_field = avg_vm1_axis_field_sg.graph_data.edge_list
output_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface/'
pathlib.Path((output_path)).mkdir(exist_ok=True)

output_path_clipped_reg = output_path+'Clipped_Surfaces/'
pathlib.Path((output_path_clipped_reg)).mkdir(exist_ok=True)

ref_pia = Surface(output_alignment_final+'/Clipped_Surfaces/Avg_vM1_Pia.vtk')
#ref_pia = Surface(input_path_avg_rf+'/Avg_vM1_Pia_50.vtk')
ref_wm = Surface(output_alignment_final+'/Clipped_Surfaces/Avg_vM1_WM.vtk')
ref_pia_wm_vm1 = Surface(output_alignment_final+'/Clipped_Surfaces/Avg_vM1_Pia.vtk')
ref_pia_wm_vm1.append(ref_wm.surface)
#print(len(avg_vm1_axis_field))
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20130320_corrected_DONE':
    pia = Surface(output_alignment_final+'Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_pia.vtk')
    wm = Surface(output_alignment_final+'Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_WM.vtk')

    pia_wm = Surface(polydata=pia.surface)
    pia_wm.append(wm.surface)

    ref_pia_wm = Surface(polydata=ref_pia.surface)
    ref_pia_wm.append(ref_wm.surface)

#     pia_wm_pts = []
#     for pt in pia.get_surface_pts():
#         pia_wm_pts.append(pt)
#     for pt in wm.get_surface_pts():
#         pia_wm_pts.append(pt)
#     ref_pia_wm_pts = []
#     for pt in ref_pia.get_surface_pts():
#         ref_pia_wm_pts.append(pt)
#     for pt in ref_wm.get_surface_pts():
#         ref_pia_wm_pts.append(pt)

#     pia_wm = Surface(pts=pia_wm_pts)
#     ref_pia_wm = Surface(pts=ref_pia_wm_pts)
    cleaned_pia_wm = clean_pdata(pia_wm.surface)
    cleaned_ref_pia_wm = clean_pdata(ref_pia_wm.surface)

    #moving_hull = Surface(polydata=cleaned_pia_wm).create_delunay_surface_3d(return_hull=True,output_filename=output_path_clipped_reg+'moving_hull.vtk')
    #target_hull = Surface(polydata=cleaned_ref_pia_wm).create_delunay_surface_3d(return_hull=True,output_filename=output_path_clipped_reg+'target_hull.vtk')

    aligner1 = Alignment(cleaned_ref_pia_wm,cleaned_pia_wm,nr_iterations=10000)
    txmat = aligner1.get_transformation_matrix()
    print(os.path.basename(file)[:-3])
    print(txmat)
    icp = aligner1.get_icp_transform()

    aligner2 = Alignment() 
    aligner2.transform_folders(output_alignment_final+'Aligned_To_MaxY/',\
                              output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    aligner3 = Alignment() 
    aligner3.transform_files(output_alignment_final+'Clipped_Surfaces/',\
                              output_path_clipped_reg,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    

In [ ]:
# Get nearest axes to the reg neuron somata
input_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface/'
avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'Avg_Axis_Field_50.am',generic_graph=True).graph_data.edge_list
nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
soma_pia_axes_sg = AmiraSpatialGraph(generic_graph=True)
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #print(i, os.path.basename(file))
    soma_center = Landmarks(input_path + '2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii').pts[0]
    neaerst_vec,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,avg_axis_field,axis_validation_distance=5000)
    nearest_axis_sg.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
    
    soma_pia_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                              +os.path.basename(file)[:-3]+'_soma_pia_axis.am',generic_graph=True).graph_data.edge_list[0]
    soma_pia_axes_sg.graph_data.add_edge(soma_pia_axis[0],soma_pia_axis[1])
nearest_axis_sg.write_spatial_graph(input_path+'Nearest_Axes.am')
soma_pia_axes_sg.write_spatial_graph(input_path+'Original_Soma_Pia_Axes.am')


### Using Clipped Surfaces Hull

In [ ]:
# avg_vm1_axis_field_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',\
#                                        generic_graph=True)
# avg_vm1_axis_field = avg_vm1_axis_field_sg.graph_data.edge_list
output_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Hull/'
pathlib.Path((output_path)).mkdir(exist_ok=True)

output_path_clipped_reg = output_path+'Clipped_Surfaces/'
pathlib.Path((output_path_clipped_reg)).mkdir(exist_ok=True)

ref_pia = Surface(output_alignment_final+'/Clipped_Surfaces/Avg_vM1_Pia.vtk')
#ref_pia = Surface(input_path_avg_rf+'/Avg_vM1_Pia_50.vtk')
ref_wm = Surface(output_alignment_final+'/Clipped_Surfaces/Avg_vM1_WM.vtk')
ref_pia_wm_vm1 = Surface(output_alignment_final+'/Clipped_Surfaces/Avg_vM1_Pia.vtk')
ref_pia_wm_vm1.append(ref_wm.surface)
#print(len(avg_vm1_axis_field))
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20130320_corrected_DONE':
    pia = Surface(output_alignment_final+'Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_pia.vtk')
    wm = Surface(output_alignment_final+'Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_WM.vtk')

    pia_wm = Surface(polydata=pia.surface)
    pia_wm.append(wm.surface)

    ref_pia_wm = Surface(polydata=ref_pia.surface)
    ref_pia_wm.append(ref_wm.surface)

#     pia_wm_pts = []
#     for pt in pia.get_surface_pts():
#         pia_wm_pts.append(pt)
#     for pt in wm.get_surface_pts():
#         pia_wm_pts.append(pt)
#     ref_pia_wm_pts = []
#     for pt in ref_pia.get_surface_pts():
#         ref_pia_wm_pts.append(pt)
#     for pt in ref_wm.get_surface_pts():
#         ref_pia_wm_pts.append(pt)

#     pia_wm = Surface(pts=pia_wm_pts)
#     ref_pia_wm = Surface(pts=ref_pia_wm_pts)
    cleaned_pia_wm = clean_pdata(pia_wm.surface)
    cleaned_ref_pia_wm = clean_pdata(ref_pia_wm.surface)

    moving_hull = Surface(polydata=cleaned_pia_wm).create_delunay_surface_3d(return_hull=True,output_filename=output_path_clipped_reg+'moving_hull.vtk')
    target_hull = Surface(polydata=cleaned_ref_pia_wm).create_delunay_surface_3d(return_hull=True,output_filename=output_path_clipped_reg+'target_hull.vtk')

    aligner1 = Alignment(target_hull,moving_hull,nr_iterations=1000)
    txmat = aligner1.get_transformation_matrix()
    print(os.path.basename(file)[:-3])
    print(txmat)
    icp = aligner1.get_icp_transform()

    aligner2 = Alignment() 
    aligner2.transform_folders(output_alignment_final+'Aligned_To_MaxY/',\
                              output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    aligner3 = Alignment() 
    aligner3.transform_files(output_alignment_final+'Clipped_Surfaces/',\
                              output_path_clipped_reg,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    

### Refine Using Original Neuron Axis and Nearest Avg Axis 

In [ ]:
# avg_vm1_axis_field_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',\
#                                        generic_graph=True)
# avg_vm1_axis_field = avg_vm1_axis_field_sg.graph_data.edge_list
# ref_pia_wm = Surface(pts=avg_vm1_axis_field_sg.graph_data.edge_pt_coords)

input_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface/'

output_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Refined_By_Neuron_Axis/'
pathlib.Path((output_path)).mkdir(exist_ok=True)

avg_vm1_axis_field_sg = AmiraSpatialGraph(input_path_avg_rf+'Avg_vM1_Axis_Field_50.am',\
                                        generic_graph=True)
avg_vm1_axis_field =  avg_vm1_axis_field_sg.graph_data.edge_list                                    
#print(len(avg_vm1_axis_field)) 
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20180109_1_Cell_A_done.am_Segmented_corrected_shifted_and_merged_DONE':
        
    print(os.path.basename(file)[:-3])
    sg = AmiraSpatialGraph(input_path+'/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
#     nearest_edge_in_avg_vm1,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
#         Vectors().get_nearest_axis_to_pt(soma_center,avg_vm1_axis_field,axis_validation_distance=3000)
    
    if os.path.exists(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am'):
        # wm proj exists... use the pia and wm to do this one now
        original_soma_pia_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                    generic_graph=True)
        original_soma_wm_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am',\
                                    generic_graph=True)
        original_neuron_axis = [original_soma_wm_axis_sg.graph_data.edge_list[0][1],\
                                original_soma_pia_axis_sg.graph_data.edge_list[0][1]]
        ref_neuron_axis = [nearest_edge_in_avg_vm1[0],nearest_edge_in_avg_vm1[1]]
    else:
        original_soma_pia_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                    generic_graph=True)
        original_neuron_axis = [original_soma_pia_axis_sg.graph_data.edge_list[0][0],\
                                original_soma_pia_axis_sg.graph_data.edge_list[0][1]]
        soma_pia_dist = Vectors().get_vec_length(original_neuron_axis)
        soma_pt_on_avg_axis = Vectors().create_pt_along_vector_at_given_distance(soma_pia_dist,nearest_edge_in_avg_vm1[1],nearest_edge_in_avg_vm1[0])
        ref_neuron_axis = [soma_pt_on_avg_axis,nearest_edge_in_avg_vm1[1]]
    
    selected_axis_sg = AmiraSpatialGraph(generic_graph=True)
    selected_axis_sg.graph_data.add_edge(nearest_edge_in_avg_vm1[0],nearest_edge_in_avg_vm1[1])
    selected_axis_sg.write_spatial_graph(output_path+'selected_axis.am')
    print(original_neuron_axis,ref_neuron_axis)
    # align oginal neuron axis to the nearest axis in the avg 
    landmarks = Landmarks(pts=original_neuron_axis)
    icp,txmat = landmarks.align_landmarks(ref_neuron_axis)
    print(txmat)
    
    aligner2 = Alignment() 
    aligner2.transform_folders(input_path,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    output_path_clipped_surf = output_path+'Clipped_Surfaces/'
    pathlib.Path((output_path_clipped_surf)).mkdir(exist_ok=True)
    aligner3 = Alignment() 
    aligner3.transform_files(input_path+'Clipped_Surfaces/',output_path_clipped_surf,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
    
    

### Refine Using Neuron Avg Axis and Nearest Local Avg Axis in Avg Surface

In [ ]:
# avg_vm1_axis_field_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',\
#                                        generic_graph=True)
# avg_vm1_axis_field = avg_vm1_axis_field_sg.graph_data.edge_list
# ref_pia_wm = Surface(pts=avg_vm1_axis_field_sg.graph_data.edge_pt_coords)

input_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface/'

output_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Refined_By_Local_Avg_Axis/'
pathlib.Path((output_path)).mkdir(exist_ok=True)

avg_vm1_axis_field_sg = AmiraSpatialGraph(input_path_avg_rf+'Avg_vM1_Axis_Field_50.am',\
                                        generic_graph=True)
avg_vm1_axis_field =  avg_vm1_axis_field_sg.graph_data.edge_list                                    
#print(len(avg_vm1_axis_field)) 
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20180109_1_Cell_A_done.am_Segmented_corrected_shifted_and_merged_DONE':
        
    print(os.path.basename(file)[:-3])
    sg = AmiraSpatialGraph(input_path+'/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    nearest_edge_in_avg_vm1,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
        Vectors().get_nearest_axis_to_pt(soma_center,avg_vm1_axis_field,axis_validation_distance=3000)
    
    if os.path.exists(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am'):
        # wm proj exists... use the pia and wm to do this one now
        original_soma_pia_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am',\
                                    generic_graph=True)
        original_soma_wm_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am',\
                                    generic_graph=True)
        original_neuron_axis = [original_soma_wm_axis_sg.graph_data.edge_list[0][1],\
                                original_soma_pia_axis_sg.graph_data.edge_list[0][1]]
        ref_neuron_axis = [nearest_edge_in_avg_vm1[0],nearest_edge_in_avg_vm1[1]]
    else:
        original_soma_pia_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                    generic_graph=True)
        original_neuron_axis = [original_soma_pia_axis_sg.graph_data.edge_list[0][0],\
                                original_soma_pia_axis_sg.graph_data.edge_list[0][1]]
        soma_pia_dist = Vectors().get_vec_length(original_neuron_axis)
        soma_pt_on_avg_axis = Vectors().create_pt_along_vector_at_given_distance(soma_pia_dist,nearest_edge_in_avg_vm1[1],nearest_edge_in_avg_vm1[0])
        ref_neuron_axis = [soma_pt_on_avg_axis,nearest_edge_in_avg_vm1[1]]
    
    selected_axis_sg = AmiraSpatialGraph(generic_graph=True)
    selected_axis_sg.graph_data.add_edge(nearest_edge_in_avg_vm1[0],nearest_edge_in_avg_vm1[1])
    selected_axis_sg.write_spatial_graph(output_path+'selected_axis.am')
    #print(original_neuron_axis,ref_neuron_axis)
    # align oginal neuron axis to the nearest axis in the avg 
    landmarks = Landmarks(pts=original_neuron_axis)
    icp,txmat = landmarks.align_landmarks(ref_neuron_axis)
    #print(txmat)
    
    print('original_neuron_axis_length : {}, neuron_axis_length_in_ref_frame : {}, Diff:{} '.format(\
            Vectors().get_vec_length(original_neuron_axis),Vectors().get_vec_length(ref_neuron_axis),
                            Vectors().get_vec_length(original_neuron_axis)-Vectors().get_vec_length(ref_neuron_axis)) )
    
    aligner2 = Alignment() 
    aligner2.transform_folders(input_path,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    output_path_clipped_surf = output_path+'Clipped_Surfaces/'
    pathlib.Path((output_path_clipped_surf)).mkdir(exist_ok=True)
    aligner3 = Alignment() 
    aligner3.transform_files(input_path+'Clipped_Surfaces/',output_path_clipped_surf,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
    
    

### Refine Using Neuron Axis and Same Axis Extended in Avg Surface

In [ ]:
# find the avg axis along the registered neuron
def extend_neuron_axis_to_avg_vm1(input_path,exp_name,ref_pia,ref_wm):
    
    #if exp_name == 'RA20180412_2_Cell_A.am_Segmented_corrected_DONE':
    print(exp_name)
    sg = AmiraSpatialGraph(input_path+'/1_Surfaces/'+exp_name+'_barrels.am')

    neuron_axis_path = input_path+'/2_Local_Neuron_Axis/'
    soma_center = Landmarks(neuron_axis_path+exp_name+'_Soma_Centroid.landmarksAscii').pts[0]
    soma_pia_axis = AmiraSpatialGraph(neuron_axis_path + exp_name + '_soma_pia_axis.am' , generic_graph=True)\
                    .graph_data.edge_list[0]
    pia_intersec_pt_reg,b = ref_pia.get_vector_intersection_pt(soma_center,soma_pia_axis[1],extrapolation_len=10000)
    print(pia_intersec_pt_reg)
    if len(pia_intersec_pt_reg)>0:
        soma_pia_axis_reg = [soma_center,pia_intersec_pt_reg]
        #write_edge(soma_pia_axis_reg,reg_axis_path+exp_name+'_soma_pia_axis_reg.am')
    else:
        pia_intersec_pt_reg,b = ref_pia.get_vector_intersection_pt(soma_center,soma_pia_axis[1],extrapolation_len=-10000)
        if len(pia_intersec_pt_reg)>0:
            soma_pia_axis_reg = [soma_center,pia_intersec_pt_reg]
        else:
            print('pia intersection not found error')

    if os.path.exists(neuron_axis_path+exp_name+'_soma_wm_axis.am') :
        soma_wm_axis = AmiraSpatialGraph(neuron_axis_path + exp_name + '_soma_wm_axis.am' , generic_graph=True)\
                            .graph_data.edge_list[0]
        wm_intersec_pt_reg,b = ref_wm.get_vector_intersection_pt(soma_center,soma_wm_axis[1],extrapolation_len=10000)
        print(wm_intersec_pt_reg)
        if len(wm_intersec_pt_reg)>0:
            soma_wm_axis_reg = [soma_center,wm_intersec_pt_reg]
            #write_edge(soma_wm_axis_reg,reg_axis_path+exp_name+'_soma_wm_axis_reg.am')
        else:
#             wm_intersec_pt_reg,b = ref_wm.get_vector_intersection_pt(soma_center,soma_wm_axis[1],extrapolation_len=-10000)
#             soma_wm_axis_reg = [soma_center,wm_intersec_pt_reg]
#         if len(wm_intersec_pt_reg)==0:
            soma_wm_axis_reg = []
            print('wm intersection not found error')            
    else:
        # No wm projection found in the original
        soma_wm_axis = []
        soma_wm_axis_reg = []

    return soma_pia_axis_reg,soma_wm_axis_reg
        

In [ ]:
# avg_vm1_axis_field_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',\
#                                        generic_graph=True)
# avg_vm1_axis_field = avg_vm1_axis_field_sg.graph_data.edge_list
# ref_pia_wm = Surface(pts=avg_vm1_axis_field_sg.graph_data.edge_pt_coords)
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_50.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_50.vtk')

input_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface/'

output_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Refined_By_Neuron_Axis_2/'
pathlib.Path((output_path)).mkdir(exist_ok=True)

avg_vm1_axis_field_sg = AmiraSpatialGraph(input_path_avg_rf+'Avg_vM1_Axis_Field_50.am',\
                                        generic_graph=True)
avg_vm1_axis_field =  avg_vm1_axis_field_sg.graph_data.edge_list                                    
#print(len(avg_vm1_axis_field)) 
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20180109_1_Cell_A_done.am_Segmented_corrected_shifted_and_merged_DONE':
        
    print(os.path.basename(file)[:-3])
    sg = AmiraSpatialGraph(input_path+'/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    
    soma_pia_axis_reg,soma_wm_axis_reg = extend_neuron_axis_to_avg_vm1(input_path,os.path.basename(file)[:-3],ref_pia,ref_wm)
    
    if os.path.exists(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am') and \
        len(soma_wm_axis_reg) > 0:
        # wm proj exists... use the pia and wm to do this one now
        original_soma_pia_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                    generic_graph=True)
        original_soma_wm_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am',\
                                    generic_graph=True)
        original_neuron_axis = [original_soma_wm_axis_sg.graph_data.edge_list[0][1],\
                                original_soma_pia_axis_sg.graph_data.edge_list[0][1]]
        ref_neuron_axis = [soma_wm_axis_reg[1],soma_pia_axis_reg[1]]
    else:
        original_soma_pia_axis_sg = AmiraSpatialGraph(input_path+'/2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                    generic_graph=True)
        original_neuron_axis = [original_soma_pia_axis_sg.graph_data.edge_list[0][0],\
                                original_soma_pia_axis_sg.graph_data.edge_list[0][1]]
        soma_pia_dist = Vectors().get_vec_length(original_neuron_axis)
        soma_pt_on_avg_axis = Vectors().create_pt_along_vector_at_given_distance(soma_pia_dist,nearest_edge_in_avg_vm1[1],nearest_edge_in_avg_vm1[0])
        ref_neuron_axis = [soma_pt_on_avg_axis,nearest_edge_in_avg_vm1[1]]
    
    selected_axis_sg = AmiraSpatialGraph(generic_graph=True)
    selected_axis_sg.graph_data.add_edge(ref_neuron_axis[0],ref_neuron_axis[1])
    selected_axis_sg.write_spatial_graph(output_path+'selected_axis.am')
    print(original_neuron_axis,ref_neuron_axis)
    # align oginal neuron axis to the nearest axis in the avg 
    landmarks = Landmarks(pts=original_neuron_axis)
    icp,txmat = landmarks.align_landmarks(ref_neuron_axis)
    print(txmat)
    
    aligner2 = Alignment() 
    aligner2.transform_folders(input_path,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)

    output_path_clipped_surf = output_path+'Clipped_Surfaces/'
    pathlib.Path((output_path_clipped_surf)).mkdir(exist_ok=True)
    aligner3 = Alignment() 
    aligner3.transform_files(input_path+'Clipped_Surfaces/',output_path_clipped_surf,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
    
    

### Using Axes Surrounding The Neuron

In [ ]:
avg_vm1_axis_field_sg = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+'Avg_vM1_Axis_Field.am',\
                                       generic_graph=True)
avg_vm1_axis_field = avg_vm1_axis_field_sg.graph_data.edge_list
ref_pia_wm = Surface(pts=avg_vm1_axis_field_sg.graph_data.edge_pt_coords)
output_path = output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Refined/'
pathlib.Path((output_path)).mkdir(exist_ok=True)
#print(len(avg_vm1_axis_field)) 
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    if os.path.basename(file)[:-3] == 'RA20180109_1_Cell_A_done.am_Segmented_corrected_shifted_and_merged_DONE':
        #pia = Surface(output_alignment_final+'Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_pia.vtk')
        #wm = Surface(output_alignment_final+'Clipped_Surfaces/'+os.path.basename(file)[:-3]+'_WM.vtk')
        sg = AmiraSpatialGraph(output_alignment_final+'Aligned_To_MaxY/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
        vm1_axis_field = AmiraSpatialGraph(output_alignment_final+'/Clipped_Surfaces/'+os.path.basename(file)[:-3]+\
                                           '_vM1_Axis_Field.am',generic_graph=True).graph_data.edge_list
        soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
        # clean up the axis field to only take the ones pointing upwards
        clean_field_sg = clean_axis_field(vm1_axis_field)

        print(os.path.basename(file)[:-3])
        
        # Get local axis field
#         nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(clean_field_sg.graph_data.edge_list,[soma_center],1000,\
#                                                                       axis_validation_xy_distance=10000)
#         nearby_axes_in_avg = Vectors().get_nearest_axes_to_pts_within_radius(avg_vm1_axis_field_sg.graph_data.edge_list,\
#                                                                              [soma_center],10000,\
#                                                                              axis_validation_xy_distance=10000,\
#                                                                             final_validation_distance=10000)
        
        print(len(nearby_axes_in_avg),len(nearby_axes))
        # pick up the equal number of axes
    #     if len(nearby_axes) > len(nearby_axes_in_avg):
    #         nearby_axes = get_number_of_axes_from_field(nearby_axes,len(nearby_axes_in_avg))
    #     elif len(nearby_axes) < len(nearby_axes_in_avg):
    #         nearby_axes_in_avg = get_number_of_axes_from_field(nearby_axes_in_avg,len(nearby_axes))

        nearby_axes_sg = get_sg_from_edge_list(nearby_axes)
        nearby_axes_in_avg_sg = get_sg_from_edge_list(nearby_axes_in_avg)

        ref_pia_wm = Surface(pts=nearby_axes_in_avg_sg.graph_data.edge_pt_coords)
        pia_wm = Surface(pts=nearby_axes_sg.graph_data.edge_pt_coords)

        aligner1 = Alignment(ref_pia_wm.surface,pia_wm.surface,nr_iterations=10000)
        txmat = aligner1.get_transformation_matrix()
        #print(txmat)
        icp = aligner1.get_icp_transform()

        aligner2 = Alignment() 
        aligner2.transform_folders(input_path,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
        
    

In [ ]:
angles = []

for edge in avg_vm1_axis_field_sg.graph_data.edge_list:
    angles.append(abs(Vectors().get_angle_between_vectors([[0,0,0],[0,0,1]],edge,ignore_opposite_direction=True)))
    

In [ ]:
np.array(angles).max(),

## Evaluate alignment

In [ ]:
def get_local_avg_axis(avg_vm1_axis_field,soma_center,pia,wm,exp_name,output_path):
    soma_wm_axis = []
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(avg_vm1_axis_field.graph_data.edge_list,[soma_center],800)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    #nearby_axes_sg.write_spatial_graph(output_neuron_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*6000 + soma_center)
    pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)
    write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
               output_path+exp_name+'_soma_pia_axis.am')
    
    wm_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = wm.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)
    if len(wm_proj_pt_neuron_using_local_avg_axis)>0:
        soma_wm_axis = [soma_center,wm_proj_pt_neuron_using_local_avg_axis]
        write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],\
               output_path+exp_name+'_soma_wm_axis.am')
    
    
    return [soma_center,pia_proj_pt_neuron_using_local_avg_axis],soma_wm_axis

In [ ]:
def get_dend_pca_axis(sg,soma_center,pia,wm,exp_name,output_path,original_neuron_axis):
    soma_wm_axis = []
    pca_components = get_pca_direction(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)
    #Landmarks(pts=resampled_pts).write_landmarks(output_root+'pts')
    end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
    #Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
    #print(end_pt)
    angle = Vectors().get_angle_between_vectors([soma_center,original_neuron_axis[1]],\
                     [soma_center,end_pt+soma_center],ignore_opposite_direction=False)
    #print(angle)
    pia_proj_pt_neuron = []
    if angle < 90:
        # pca direction is correct.. project to pia
        end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
        print(end_pt)
        Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
        pia_proj_pt_neuron,soma_depth  = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
        write_edge([soma_center,pia_proj_pt_neuron],output_path+exp_name+'_soma_pia_axis.am')
    else:
        # the pca is inverted, so straighten it
        #print('opposite')
        end_pt = Vectors().create_pt_along_vector_at_given_distance(-6000,[0,0,0],pca_components[0,:])
        pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
        write_edge([soma_center,pia_proj_pt_neuron],output_path+exp_name+'_soma_pia_axis.am')

    wm_proj_pt_neuron = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron)
    if len (wm_proj_pt_neuron) > 0:
        write_edge([soma_center,wm_proj_pt_neuron],output_path+exp_name+'_soma_wm_axis.am')
    
    
    return [soma_center,pia_proj_pt_neuron_using_local_avg_axis],soma_wm_axis

In [ ]:
# find the avg axis along the registered neuron
def get_registered_neuron_stats(input_path,ref_pia,ref_wm):
    cols = ['Exp_Name','Neuron_Depth_Original','Neuron_Depth_Registered',\
        'Neuron_Height_Original','Neuron_Height_Registered',\
        'Neuron_Cortical_Thickness_Original','Neuron_Cortical_Thickness_Registered',\
        'Neuron_Height_Original','Neuron_Height_Registered',\
        'Neuron_Depth_Difference','Neuron_Height_Difference','Neuron_Cortical_Thickness_Difference',\
        'Neuron_Depth_Difference_Abs','Neuron_Height_Difference_Abs','Neuron_Cortical_Thickness_Difference_Abs']
    df_main = pd.DataFrame(columns=cols)
    
    reg_axis_path = input_path+'/Reg_Neuron_Axis/'
    pathlib.Path((reg_axis_path)).mkdir(exist_ok=True)
    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        exp_name = os.path.basename(file)[:-3]
        #if exp_name == 'RA20180412_2_Cell_A.am_Segmented_corrected_DONE':
        print(exp_name)
        sg = AmiraSpatialGraph(input_path+'/1_Surfaces/'+exp_name+'_barrels.am')

        neuron_axis_path = input_path+'/2_Local_Neuron_Axis/'
        soma_center = Landmarks(neuron_axis_path+exp_name+'_Soma_Centroid.landmarksAscii').pts[0]
        soma_pia_axis = AmiraSpatialGraph(neuron_axis_path + exp_name + '_soma_pia_axis.am' , generic_graph=True)\
                        .graph_data.edge_list[0]
        pia_intersec_pt_reg,b = ref_pia.get_vector_intersection_pt(soma_center,soma_pia_axis[1],extrapolation_len=10000)
        print(pia_intersec_pt_reg)
        if len(pia_intersec_pt_reg)>0:
            soma_pia_axis_reg = [soma_center,pia_intersec_pt_reg]
            write_edge(soma_pia_axis_reg,reg_axis_path+exp_name+'_soma_pia_axis_reg.am')
        else:
            soma_pia_axis_reg = []
            print('pia intersection not found error')

        if os.path.exists(neuron_axis_path+exp_name+'_soma_wm_axis.am') :
            soma_wm_axis = AmiraSpatialGraph(neuron_axis_path + exp_name + '_soma_wm_axis.am' , generic_graph=True)\
                                .graph_data.edge_list[0]
            wm_intersec_pt_reg,b = ref_wm.get_vector_intersection_pt(soma_center,soma_wm_axis[1],extrapolation_len=10000)
            print(wm_intersec_pt_reg)
            if len(wm_intersec_pt_reg)>0:
                soma_wm_axis_reg = [soma_center,wm_intersec_pt_reg]
                write_edge(soma_wm_axis_reg,reg_axis_path+exp_name+'_soma_wm_axis_reg.am')
            else:
    #             wm_intersec_pt_reg,b = ref_wm.get_vector_intersection_pt(soma_center,soma_wm_axis[1],extrapolation_len=-10000)
    #             soma_wm_axis_reg = [soma_center,wm_intersec_pt_reg]
    #         if len(wm_intersec_pt_reg)==0:
                soma_wm_axis_reg = []
                print('wm intersection not found error')            
        else:
            # No wm projection found in the original
            soma_wm_axis = []
            soma_wm_axis_reg = []


        # get the difference between original and other depth
        df = pd.DataFrame(columns=cols)
        df['Exp_Name'] = [exp_name]
        if len(soma_pia_axis)>0:
            df['Neuron_Depth_Original'] = [Vectors().get_vec_length(soma_pia_axis)]
        if len(soma_pia_axis_reg)>0:
            df['Neuron_Depth_Registered'] = [Vectors().get_vec_length(soma_pia_axis_reg)]
        if len(soma_wm_axis)>0:
            df['Neuron_Height_Original'] = [Vectors().get_vec_length(soma_wm_axis)]
            df['Neuron_Cortical_Thickness_Original'] = df['Neuron_Depth_Original'].values+df['Neuron_Height_Original'].values
        if len(soma_wm_axis_reg) > 0:
            df['Neuron_Height_Registered'] = [Vectors().get_vec_length(soma_wm_axis_reg)]
            df['Neuron_Cortical_Thickness_Registered'] = df['Neuron_Depth_Registered'].values+df['Neuron_Height_Registered'].values
        df['Neuron_Depth_Difference'] = df['Neuron_Depth_Original'].values-df['Neuron_Depth_Registered'].values
        df['Neuron_Height_Difference'] = df['Neuron_Height_Original'].values-df['Neuron_Height_Registered'].values
        df['Neuron_Cortical_Thickness_Difference'] = df['Neuron_Cortical_Thickness_Original'].values-df['Neuron_Cortical_Thickness_Registered'].values
        df['Neuron_Depth_Difference_Abs'] = abs(df['Neuron_Depth_Difference'])
        df['Neuron_Height_Difference_Abs'] = abs(df['Neuron_Height_Difference'])
        df['Neuron_Cortical_Thickness_Difference_Abs'] = abs(df['Neuron_Cortical_Thickness_Difference'])

        df_main = df_main.append(df)

        
    df_main.to_csv(input_path+'Depth_Difference.csv')
    
    return df_main

In [ ]:
#avg_vm1_axis_field = AmiraSpatialGraph(output_alignment_final+'Clipped_Surfaces/Avg_vM1_Axis_Field.am',generic_graph=True)
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_50.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_50.vtk')

df_reg_clipped_surf_refined_local_avg = get_registered_neuron_stats(output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Refined_By_Local_Avg_Axis/',\
                                     ref_pia,ref_wm)
# df_reg_hull = get_registered_neuron_stats(output_alignment_final+'Aligned_To_Clipped_Avg_Surface_Hull/',\
#                                      ref_pia,ref_wm)
# df_reg_full_surf = get_registered_neuron_stats(output_root+'3_Aligned_By_Surfaces/',\
#                                      ref_pia,ref_wm)
#df_reg_refined.to_csv(output_path+'Reg_Refined_Stats.csv')

In [ ]:
#df_reg_hull[(df_reg_refined['Neuron_Depth_Difference_Abs']>300)]
#df_reg_hull.dropna()
#df_reg_clipped_surf.dropna()
#df_reg_hull.sort_values(by='Neuron_Depth_Difference_Abs')
#(df_reg_hull[(df_reg_hull['Neuron_Depth_Difference_Abs']>200)])
#(df_reg_refined[(df_reg_refined['Neuron_Depth_Difference_Abs']>300)])
#(df_reg_clipped_surf_refined[(df_reg_clipped_surf_refined['Neuron_Depth_Difference_Abs']>400)])
#df_reg_clipped_surf_refined['Neuron_Height_Difference_Abs'].mean()
#df_reg_refined['Neuron_Depth_Difference_Abs'].mean(),df_reg_refined['Neuron_Depth_Difference_Abs'].std()
#df_reg_hull['Neuron_Depth_Difference_Abs'].mean(),df_reg_hull['Neuron_Depth_Difference_Abs'].std()
#df_reg_full_surf['Neuron_Depth_Difference_Abs'].mean(),df_reg_full_surf['Neuron_Depth_Difference_Abs'].std()
df_reg_clipped_surf_refined_local_avg['Neuron_Cortical_Thickness_Difference_Abs'].mean(),df_reg_clipped_surf_refined_local_avg['Neuron_Cortical_Thickness_Difference_Abs'].std()

In [ ]:
#df_reg_clipped_surf_refined[df_reg_clipped_surf_refined['Neuron_Cortical_Thickness_Difference_Abs'] > 500]
dfref_surf = pd.read_csv(output_alignment_final+'Aligned_To_Clipped_Avg_Surface/Depth_Difference.csv')
dfref_surf['Neuron_Cortical_Thickness_Original'].mean(),dfref_surf['Neuron_Cortical_Thickness_Original'].std(),\
dfref_surf['Neuron_Cortical_Thickness_Registered'].mean(),dfref_surf['Neuron_Cortical_Thickness_Registered'].std()

# Write Registered Neurons

In [ ]:
## Combine Neurons for visualization
apical = AmiraSpatialGraph()
basal = AmiraSpatialGraph()
dend = AmiraSpatialGraph()
#axon = AmiraSpatialGraph()
neuron = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    sg = AmiraSpatialGraph(output_alignment_final+'Aligned_To_Clipped_Avg_Surface/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
        apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
    if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
        basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])
    
neuron.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1_Using_Clipped_Surface.am')
apical.write_spatial_graph(output_root+'All_Apical_Dendrites_Registered_To_vM1_Using_Clipped_Surface.am')
basal.write_spatial_graph(output_root+'All_Basal_Dendrites_Registered_To_vM1_Using_Clipped_Surface.am')

In [ ]:
apical.write_spatial_graph(output_root+'All_Apical_Dendrites_Registered_To_vM1.am')
basal.write_spatial_graph(output_root+'All_Basal_Dendrites_Registered_To_vM1.am')

In [ ]:
apical = AmiraSpatialGraph()
basal = AmiraSpatialGraph()
dend = AmiraSpatialGraph()
#axon = AmiraSpatialGraph()
neuron = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    sg = AmiraSpatialGraph(output_root+'3_Aligned_By_Surfaces/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
        apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
    if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
        basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])
    
neuron.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1_Using_Surfaces.am')
apical.write_spatial_graph(output_root+'All_Apical_Dendrites_Registered_To_vM1_Using_Surfaces.am')
basal.write_spatial_graph(output_root+'All_Basal_Dendrites_Registered_To_vM1_Using_Surfaces.am')

In [ ]:
apical = AmiraSpatialGraph()
basal = AmiraSpatialGraph()
dend = AmiraSpatialGraph()
#axon = AmiraSpatialGraph()
neuron = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    sg = AmiraSpatialGraph(output_root+'3_Aligned_By_Barrels/1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
    neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
        apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
    if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
        basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])
    
neuron.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1_Using_Barrels.am')
apical.write_spatial_graph(output_root+'All_Apical_Dendrites_Registered_To_vM1_Using_Barrels.am')
basal.write_spatial_graph(output_root+'All_Basal_Dendrites_Registered_To_vM1_Using_Barrels.am')

# Comapre different reg methods




In [ ]:
# Find difference between manual and auto neuron axis wr.t local avg axis

cols = ['Exp','Angle_Between_Manual_and_Auto_Neuron_Axis',\
        'Angle_Between_Manual_and_Local_Avg_Axis',\
        'Angle_Between_Auto_and_Local_Avg_Axis',\
        'Angle_Between_Manual_Surface_Reg_Neuron_and_RF_Nearest_Axis',\
        'Angle_Between_Auto_Surface_Reg_Neuron_and_RF_Nearest_Axis'\
       ]
df_main = pd.DataFrame(columns=cols)

for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    
    print(os.path.basename(file))
    #sg = AmiraSpatialGraph(file,barrel_projections_present=False)
    manual_soma_pia_axis = AmiraSpatialGraph(manual_neuron_axis_path+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    neuron_soma_pia_axis = AmiraSpatialGraph(output_root+'2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    neuron_soma_pia_axis_local_avg = AmiraSpatialGraph(output_root+'2_Local_Neuron_Axis/'+\
                                             os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    manual_soma_pia_axis_surf_reg = AmiraSpatialGraph(output_alignment_using_surfaces+'2_Manual_Neuron_Axis/' + \
                                                      os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    neuron_soma_pia_axis_surf_reg = AmiraSpatialGraph(output_alignment_using_surfaces+'2_Local_Neuron_Axis/'+ \
                                                      os.path.basename(file)[:-3]+'_soma_pia_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    nearest_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'Nearest_Axis/'+ \
                                                      os.path.basename(file)[:-3]+'_nearest_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
    df = pd.DataFrame(columns=cols)
    df['Exp'] = [os.path.basename(file)[:-3]]
    df['Angle_Between_Manual_and_Auto_Neuron_Axis'] = Vectors().get_angle_between_vectors(manual_soma_pia_axis,\
                                                                                           neuron_soma_pia_axis,\
                                                                                           ignore_opposite_direction=True)
    df['Angle_Between_Manual_and_Local_Avg_Axis'] = Vectors().get_angle_between_vectors(manual_soma_pia_axis,\
                                                                                           neuron_soma_pia_axis_local_avg,\
                                                                                           ignore_opposite_direction=True)
    df['Angle_Between_Auto_and_Local_Avg_Axis'] = Vectors().get_angle_between_vectors(neuron_soma_pia_axis,\
                                                                                           neuron_soma_pia_axis_local_avg,\
                                                                                           ignore_opposite_direction=True)
    
    df['Angle_Between_Manual_Surface_Reg_Neuron_and_RF_Nearest_Axis'] = Vectors().get_angle_between_vectors(manual_soma_pia_axis_surf_reg,\
                                                                                           nearest_axis,\
                                                                                           ignore_opposite_direction=True)
    df['Angle_Between_Auto_Surface_Reg_Neuron_and_RF_Nearest_Axis'] = Vectors().get_angle_between_vectors(neuron_soma_pia_axis_surf_reg,\
                                                                                           nearest_axis,\
                                                                                           ignore_opposite_direction=True) 
    df['Nearest_Axis_Length'] = Vectors().get_vec_length(nearest_axis)
    # read soma wm axis if exits
    if os.path.exists(output_alignment_using_surfaces+'2_Manual_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am'):
        manual_soma_wm_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'2_Manual_Neuron_Axis/'+os.path.basename(file)[:-3]+\
                                                '_soma_wm_axis.am',generic_graph=True).graph_data.edge_list[0]
        
        df['Manual_Axis_Length'] = Vectors().get_vec_length(manual_soma_pia_axis) + Vectors().get_vec_length(manual_soma_wm_axis)
        
        df['Length_Difference_Between_Manual_And_Nearest_Neuron_Axis'] = df['Manual_Axis_Length'] - df['Nearest_Axis_Length'] 
        
    if os.path.exists(output_alignment_using_surfaces+'2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am'):
        neuron_soma_wm_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+\
                                                '_soma_wm_axis.am',generic_graph=True).graph_data.edge_list[0]
        
        df['Auto_Axis_Length'] = Vectors().get_vec_length(neuron_soma_pia_axis) + Vectors().get_vec_length(neuron_soma_wm_axis)
    
        df['Length_Difference_Between_Auto_And_Nearest_Neuron_Axis'] = df['Auto_Axis_Length'] - df['Nearest_Axis_Length']
    
    
    df_main = df_main.append(df)
    
df_main.to_csv(output_root+'Neuron_Axis_Difference_Original.csv')
    

In [ ]:
df_main.dropna()
df_main.mean(),df_main.std(),df_main.max()
df_main['Change_In_Manual_Angle_After_Registration'] = df_main['Angle_Between_Manual_Surface_Reg_Neuron_and_RF_Nearest_Axis']- df_main['Angle_Between_Manual_and_Local_Avg_Axis']
df_main['Change_In_Auto_Angle_After_Registration'] = df_main['Angle_Between_Auto_Surface_Reg_Neuron_and_RF_Nearest_Axis']- df_main['Angle_Between_Auto_and_Local_Avg_Axis']

df_main['Angle_Between_Auto_and_Local_Avg_Axis'].mean(),df_main['Angle_Between_Auto_and_Local_Avg_Axis'].std()

In [ ]:
df_main['Angle_Between_Auto_Surface_Reg_Neuron_and_RF_Nearest_Axis'].mean(),df_main['Angle_Between_Auto_Surface_Reg_Neuron_and_RF_Nearest_Axis'].std()

In [ ]:
(df_main['Angle_Between_Auto_Surface_Reg_Neuron_and_RF_Nearest_Axis'] - df_main['Angle_Between_Auto_and_Local_Avg_Axis'])

In [ ]:
abs(df_main['Length_Difference_Between_Manual_And_Nearest_Neuron_Axis']).mean(),abs(df_main['Length_Difference_Between_Manual_And_Nearest_Neuron_Axis']).std()

In [ ]:
abs(df_main['Length_Difference_Between_Auto_And_Nearest_Neuron_Axis']).mean(),abs(df_main['Length_Difference_Between_Auto_And_Nearest_Neuron_Axis']).std()